<a href="https://colab.research.google.com/github/HEOJY88/IMK-AI-POC/blob/main/IMK_PLUS_%ED%92%88%EB%AA%A9%EA%B4%80%EB%A0%A8_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 전처리 및 분석에 필요한 라이브러리 설치
!pip install pandas
!pip install scikit-learn
!pip install sentence-transformers
!pip install chromadb
!pip install gradio

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import glob
import os

# 파일들이 있는 폴더 경로
folder_path = '/content/drive/MyDrive/ai_poc_project/zipcode_data/'

# 모든 .txt 파일의 경로를 리스트로 가져오기
file_paths = glob.glob(folder_path + '*.txt')

# 모든 데이터프레임을 담을 빈 리스트
df_list = []

for file_path in file_paths:
    # 한글 인코딩을 지정하여 파일 읽기
    try:
        df = pd.read_csv(file_path, sep='|', encoding='utf-8', header=0, usecols=['우편번호', '시도', '시군구'])
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, sep='|', encoding='cp949', header=0, usecols=['우편번호', '시도', '시군구'])

    # 우편번호 컬럼을 정수형으로 변환하여 오류 방지
    df['우편번호'] = df['우편번호'].astype(str)

    df_list.append(df)

print("모든 파일 로드 완료.")

# 모든 데이터프레임을 하나로 합치기
zipcode_df = pd.concat(df_list, ignore_index=True)

# 중복 데이터 제거 (우편번호 파일에는 중복이 있을 수 있으므로 제거)
zipcode_df.drop_duplicates(subset=['우편번호'], inplace=True)
zipcode_df.reset_index(drop=True, inplace=True)

print(f"\n최종 합쳐진 우편번호 데이터 개수: {len(zipcode_df)} 행")

# 최종 데이터프레임을 CSV 파일로 저장
# 파일명: all_zipcodes.csv
output_file_path = os.path.join(folder_path, 'all_zipcodes.csv')
zipcode_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"\n파일이 성공적으로 저장되었습니다: {output_file_path}")

In [ ]:
import pandas as pd
import os

# 파일 저장 경로 설정
output_path = '/content/drive/MyDrive/ai_poc_project/dummy_data/'

# 폴더가 없으면 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"폴더가 생성되었습니다: {output_path}")

# 1. 주문 및 배송 더미 데이터 저장
orders_df.to_csv(os.path.join(output_path, 'dummy_orders.csv'), index=False, encoding='utf-8-sig')

# 2. 품목 정보 더미 데이터 저장
items_df.to_csv(os.path.join(output_path, 'dummy_items.csv'), index=False, encoding='utf-8-sig')

# 3. 우편번호-지역 매핑 더미 데이터 저장
# 실제 데이터를 사용할 경우 이 부분은 건너뛰고 실제 파일 경로를 사용합니다.
zipcode_df.to_csv(os.path.join(output_path, 'dummy_zipcode_mapping.csv'), index=False, encoding='utf-8-sig')

print("\n더미 데이터가 CSV 파일로 저장되었습니다.")

In [ ]:
import pandas as pd
import os

# ======================================================================
# 1. 데이터 불러오기
# ======================================================================

# 더미 데이터 경로 (혹은 실제 주문/품목 파일 경로로 수정)
dummy_data_path = '/content/drive/MyDrive/ai_poc_project/dummy_data/'
orders_df = pd.read_csv(os.path.join(dummy_data_path, 'dummy_orders.csv'))
items_df = pd.read_csv(os.path.join(dummy_data_path, 'dummy_items.csv'))

# 실제 우편번호 데이터 경로 (이전에 합친 최종 파일 이름으로 수정)
real_data_path = '/content/drive/MyDrive/ai_poc_project/zipcode_data/'
zipcode_df = pd.read_csv(os.path.join(real_data_path, 'all_zipcodes.csv'))

print("데이터 로드 완료.")

# ======================================================================
# 2. 주문 데이터와 우편번호 데이터 결합
# ======================================================================

# orders_df와 zipcode_df를 '우편번호' 컬럼을 기준으로 합치기
# how='left'를 사용해 모든 주문 데이터를 유지
merged_df = pd.merge(orders_df, zipcode_df, on='우편번호', how='left')

# 결합 후 새로운 컬럼('시도', '시군구')이 추가되었는지 확인
print("\n결합된 데이터프레임의 상위 5개 행:")
print(merged_df.head())

# ======================================================================
# 3. 새로운 컬럼 생성 (리드 타임, 주문 요일)
# ======================================================================

# 날짜 컬럼을 datetime 형식으로 변환
merged_df['주문일자'] = pd.to_datetime(merged_df['주문일자'])
merged_df['배송완료일자'] = pd.to_datetime(merged_df['배송완료일자'])

# 리드 타임(배송 소요 시간) 계산 (단위: 일)
merged_df['리드 타임'] = (merged_df['배송완료일자'] - merged_df['주문일자']).dt.days

# 주문 요일(0: 월요일, 6: 일요일) 계산
merged_df['주문 요일'] = merged_df['주문일자'].dt.dayofweek

# 최종 데이터프레임의 상위 5개 행과 컬럼 목록 다시 확인
print("\n최종 전처리된 데이터프레임의 상위 5개 행:")
print(merged_df.head())
print("\n최종 전처리된 데이터프레임의 컬럼 목록:")
print(merged_df.columns)

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os

# ======================================================================
# 1. 필요한 라이브러리 설치
# ======================================================================
# Colab 세션을 다시 시작했다면 아래 코드를 실행해주세요
!pip install -q chromadb sentence-transformers

# ======================================================================
# 2. 데이터 불러오기
# ======================================================================

# items_df 불러오기 (상품명, 상품설명을 위해 필요)
items_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_items.csv')

# orders_df 불러오기 (부서 정보(주문자소속팀)를 위해 필요)
# 수정된 더미 주문 데이터 파일을 사용합니다.
orders_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_orders.csv')


# ======================================================================
# 3. AI 임베딩 모델 로드 및 임베딩 생성
# ======================================================================

# 한국어에 특화된 경량 임베딩 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 품목 정보 텍스트 결합 (임베딩을 위한 입력 텍스트 생성)
items_df['embedding_text'] = items_df['품명'] + ' ' + items_df['규격'] + ' ' + items_df['모델명'] + ' ' + items_df['제조원'] + ' ' + items_df['상품설명']

# 품목별 임베딩 생성
item_embeddings = model.encode(items_df['embedding_text'].tolist(), show_progress_bar=True)
print(f"생성된 품목 임베딩의 형태: {item_embeddings.shape}")

# 부서 임베딩 생성 (유사 부서 분석을 위해)
# orders_df에서 '주문자소속팀' 컬럼을 사용해 유일한 부서 목록을 가져옵니다.
teams = orders_df['주문자소속팀'].unique().tolist()
team_embeddings = model.encode(teams)
print(f"생성된 부서 임베딩의 형태: {team_embeddings.shape}")


# ======================================================================
# 4. ChromaDB에 임베딩 저장
# ======================================================================

# ChromaDB 클라이언트 및 컬렉션 생성 (로컬에서 실행)
client = chromadb.Client()

# 품목 임베딩을 위한 컬렉션 생성
item_collection = client.get_or_create_collection("item_embeddings")
item_collection.add(
    embeddings=item_embeddings.tolist(),
    metadatas=[{"품목코드": code} for code in items_df['품목코드'].tolist()],
    documents=items_df['embedding_text'].tolist(),
    ids=items_df['품목코드'].tolist()
)

print("\nChromaDB에 품목 임베딩 저장 완료.")
print(f"ChromaDB에 저장된 데이터 수: {item_collection.count()}")

# 부서 임베딩을 위한 컬렉션 생성
team_collection = client.get_or_create_collection("team_embeddings")
team_collection.add(
    embeddings=team_embeddings.tolist(),
    documents=teams,
    ids=teams
)

print("\nChromaDB에 부서 임베딩 저장 완료.")
print(f"ChromaDB에 저장된 데이터 수: {team_collection.count()}")

In [ ]:
import chromadb

# ChromaDB 클라이언트 생성
client = chromadb.Client()

# 품목 임베딩 컬렉션 로드
item_collection = client.get_or_create_collection("item_embeddings")

# 부서 임베딩 컬렉션 로드
team_collection = client.get_or_create_collection("team_embeddings")


print("===========================")
print("  품목 임베딩 컬렉션 확인")
print(f"  총 데이터 개수: {item_collection.count()}개")
print("===========================\n")

# 품목 컬렉션의 상위 5개 데이터 조회
item_results = item_collection.get(limit=5)

for i in range(len(item_results['ids'])):
    print(f"ID: {item_results['ids'][i]}")
    print(f"  메타데이터: {item_results['metadatas'][i]}")
    print(f"  문서(원본 텍스트): {item_results['documents'][i]}")
    print("---------------------------------")


print("\n===========================")
print("  부서 임베딩 컬렉션 확인")
print(f"  총 데이터 개수: {team_collection.count()}개")
print("===========================\n")

# 부서 컬렉션의 모든 데이터 조회
team_results = team_collection.get(ids=team_collection.get()['ids'])

for i in range(len(team_results['ids'])):
    print(f"ID: {team_results['ids'][i]}")
    print(f"  문서(원본 텍스트): {team_results['documents'][i]}")
    print("---------------------------------")

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import gradio as gr
import numpy as np
import os
import traceback

# ======================================================================
# 2. 데이터 불러오기 및 전처리
# ======================================================================
# orders_df 불러오기 (파일명을 'dummy_orders.csv'로 수정)
orders_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_orders.csv')

# items_df 불러오기
items_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_items.csv')

# 품목코드와 품명을 매핑하는 딕셔너리 생성
item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()

# ======================================================================
# 3. AI 임베딩 생성 및 벡터 DB 구축
# ======================================================================
# 한국어 임베딩 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 품목 임베딩 생성
items_df['embedding_text'] = items_df['품명'] + ' ' + items_df['규격'] + ' ' + items_df['모델명'] + ' ' + items_df['제조원'] + ' ' + items_df['상품설명']
item_embeddings = model.encode(items_df['embedding_text'].tolist(), show_progress_bar=True)

# 부서 임베딩 생성
teams = orders_df['주문자소속팀'].unique().tolist()
team_embeddings = model.encode(teams)

# ChromaDB 클라이언트 및 컬렉션 생성
client = chromadb.Client()

# 기존 컬렉션이 존재하면 삭제 후 새로 생성 (가장 확실한 초기화 방법)
try:
    client.delete_collection(name="item_embeddings")
    client.delete_collection(name="team_embeddings")
except Exception:
    pass

item_collection = client.get_or_create_collection("item_embeddings")
team_collection = client.get_or_create_collection("team_embeddings")


# 품목 임베딩 저장
item_collection.add(
    embeddings=item_embeddings.tolist(),
    metadatas=[{"품목코드": code} for code in items_df['품목코드'].tolist()],
    documents=items_df['embedding_text'].tolist(),
    ids=items_df['품목코드'].tolist()
)

# 부서 임베딩 저장
team_collection.add(
    embeddings=team_embeddings.tolist(),
    documents=teams,
    ids=teams
)
print("벡터 DB에 품목 및 부서 임베딩 저장 완료.")

# ======================================================================
# 4. 분석 함수 정의 및 Gradio UI 구현
# ======================================================================
def calculate_dot_product_similarity(query_embedding: list, result_embeddings: list):
    """
    쿼리 임베딩과 결과 임베딩 간의 내적 유사도를 계산하는 함수
    """
    query_vector = np.array(query_embedding)
    result_vectors = np.array(result_embeddings)
    dot_product_scores = np.dot(result_vectors, query_vector)
    return dot_product_scores.tolist()


def analyze_team_preferences(target_team: str):
    """
    내적 유사도 방식을 적용하여 유사 부서를 분석하는 함수
    """
    try:
        # 데이터 로드
        orders_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_orders.csv')
        items_df = pd.read_csv('/content/drive/MyDrive/ai_poc_project/dummy_data/dummy_items.csv')
        item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()
        client = chromadb.Client()
        team_collection = client.get_or_create_collection("team_embeddings")

        team_sales = orders_df[orders_df['주문자소속팀'] == target_team]
        if team_sales.empty:
            return f"[{target_team}]의 판매 데이터가 없습니다.", "분석할 유사 부서를 찾을 수 없습니다."

        top_items = team_sales.groupby('품목코드')['수량'].sum().reset_index()
        top_items['품명'] = top_items['품목코드'].map(item_code_to_name)
        top_items = top_items.sort_values(by='수량', ascending=False).head(5)

        ranking_text = f"**[{target_team}] 품목 판매 랭킹 (Top 5)**\n"
        for _, row in top_items.iterrows():
            ranking_text += f"- {row['품명']} ({row['수량']}개)\n"

        # 벡터 DB에서 타겟 부서의 임베딩 가져오기
        target_embedding_result = team_collection.get(ids=[target_team], include=['embeddings'])
        if len(target_embedding_result['embeddings']) == 0:
            return ranking_text, "해당 부서의 임베딩이 존재하지 않습니다."

        target_embedding = np.array(target_embedding_result['embeddings'][0])

        # 유사한 부서 찾기 (가장 가까운 3개 부서, 자신 포함)
        similar_teams_result = team_collection.query(
            query_embeddings=[target_embedding.tolist()],
            n_results=4,
            include=['documents', 'embeddings']
        )

        # 문서(원본 텍스트)와 임베딩 정보 추출
        results = similar_teams_result['documents'][0]
        result_embeddings = similar_teams_result['embeddings'][0]

        # --- 내적 유사도 계산 및 정렬 ---
        dot_product_scores = calculate_dot_product_similarity(target_embedding.tolist(), result_embeddings)

        scored_results = list(zip(dot_product_scores, results))
        scored_results.sort(key=lambda x: x[0], reverse=True)

        similar_teams = [doc for score, doc in scored_results if doc != target_team]
        similar_teams = similar_teams[:3] # 상위 3개만 선택

        # --- 수정 끝 ---

        if not similar_teams:
            similar_text = "유사 부서를 찾을 수 없습니다."
        else:
            similar_teams_df = orders_df[orders_df['주문자소속팀'].isin(similar_teams)]

            if similar_teams_df.empty:
                similar_text = f"유사 부서: {', '.join(similar_teams)}\n\n유사 부서들의 판매 데이터가 충분하지 않습니다."
            else:
                similar_top_items = similar_teams_df.groupby('품목코드')['수량'].sum().reset_index()
                similar_top_items['품명'] = similar_top_items['품목코드'].map(item_code_to_name)
                similar_top_items = similar_top_items.sort_values(by='수량', ascending=False).head(5)

                similar_text = f"**[{target_team}]과 유사한 부서들의 선호 품목 (Top 5)**\n"
                similar_text += f"(유사 부서: {', '.join(similar_teams)})\n"
                for _, row in similar_top_items.iterrows():
                    similar_text += f"- {row['품명']} ({row['수량']}개)\n"
    except Exception as e:
        similar_text = f"유사 부서 분석 중 오류 발생:\n{traceback.format_exc()}"

    return ranking_text, similar_text

# Gradio UI 구현
teams = orders_df['주문자소속팀'].unique().tolist()
teams.sort()

with gr.Blocks() as demo:
    gr.Markdown("# PoC #1: 부서별 품목 선호도 분석")
    gr.Markdown("특정 부서를 선택하면, 해당 부서의 판매 랭킹과 AI가 추천하는 유사 부서의 선호 품목을 분석합니다.")

    with gr.Row():
        team_dropdown = gr.Dropdown(choices=teams, label="분석할 부서를 선택하세요", value=teams[0])
        run_button = gr.Button("분석 실행")

    with gr.Row():
        ranking_output = gr.Textbox(label="선택한 부서의 품목 판매 랭킹", lines=6)
        similar_output = gr.Textbox(label="유사 부서들의 선호 품목", lines=6)

    run_button.click(
        fn=analyze_team_preferences,
        inputs=team_dropdown,
        outputs=[ranking_output, similar_output]
    )

if __name__ == "__main__":
    demo.launch(share=True)

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import gradio as gr
import numpy as np
import os
import traceback

# ======================================================================
# 1. 초기 설정 및 라이브러리 설치
# ======================================================================
# 요구사항 파일을 생성하고 설치하여 환경을 일관되게 유지
with open("requirements.txt", "w") as f:
    f.write("pandas\nchromadb\nsentence-transformers\ngradio\n")
!pip install -r requirements.txt
from google.colab import drive
drive.mount('/content/drive')

# ======================================================================
# 2. 데이터 불러오기 및 전처리
# ======================================================================
# 파일 경로 설정
DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
DATA_PATH = os.path.join(DRIVE_PATH, 'dummy_data')
DB_PATH = os.path.join(DRIVE_PATH, 'db_data')

# 데이터 불러오기
orders_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_orders.csv'))
items_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_items.csv'))

# 품목코드와 품명을 매핑하는 딕셔너리 생성
item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()

# ======================================================================
# 3. AI 임베딩 생성 및 벡터 DB 구축 (지속성 모드 적용)
# ======================================================================
# ChromaDB 클라이언트를 지속성 모드로 설정
client = chromadb.PersistentClient(path=DB_PATH)

# 컬렉션 존재 여부 확인 후 데이터 로드
try:
    item_collection = client.get_collection("item_embeddings")
    team_collection = client.get_collection("team_embeddings")
    print("기존 벡터 DB 컬렉션을 성공적으로 불러왔습니다.")

except ValueError:
    print("새로운 벡터 DB를 구축합니다.")
    # 컬렉션이 없으면 새로 생성
    item_collection = client.get_or_create_collection("item_embeddings")
    team_collection = client.get_or_create_collection("team_embeddings")

    # 한국어 임베딩 모델 로드
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')

    # 품목 임베딩 생성
    items_df['embedding_text'] = items_df['품명'] + ' ' + items_df['규격'] + ' ' + items_df['모델명'] + ' ' + items_df['제조원'] + ' ' + items_df['상품설명']
    item_embeddings = model.encode(items_df['embedding_text'].tolist(), show_progress_bar=True)

    # 부서 임베딩 생성
    teams = orders_df['주문자소속팀'].unique().tolist()
    team_embeddings = model.encode(teams)

    # 품목 임베딩 저장
    item_collection.add(
        embeddings=item_embeddings.tolist(),
        metadatas=[{"품목코드": code} for code in items_df['품목코드'].tolist()],
        documents=items_df['embedding_text'].tolist(),
        ids=items_df['품목코드'].tolist()
    )

    # 부서 임베딩 저장
    team_collection.add(
        embeddings=team_embeddings.tolist(),
        documents=teams,
        ids=teams
    )
    print("새로운 벡터 DB에 품목 및 부서 임베딩 저장 완료.")

# ======================================================================
# 4. 분석 함수 정의 및 Gradio UI 구현
# ======================================================================
def calculate_dot_product_similarity(query_embedding: list, result_embeddings: list):
    """
    쿼리 임베딩과 결과 임베딩 간의 내적 유사도를 계산하는 함수
    """
    query_vector = np.array(query_embedding)
    result_vectors = np.array(result_embeddings)
    dot_product_scores = np.dot(result_vectors, query_vector)
    return dot_product_scores.tolist()


def analyze_team_preferences(target_team: str):
    """
    내적 유사도 방식을 적용하여 유사 부서를 분석하는 함수
    """
    try:
        # 데이터 로드
        orders_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_orders.csv'))
        items_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_items.csv'))
        item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()

        team_sales = orders_df[orders_df['주문자소속팀'] == target_team]
        if team_sales.empty:
            return f"[{target_team}]의 판매 데이터가 없습니다.", "분석할 유사 부서를 찾을 수 없습니다."

        top_items = team_sales.groupby('품목코드')['수량'].sum().reset_index()
        top_items['품명'] = top_items['품목코드'].map(item_code_to_name)
        top_items = top_items.sort_values(by='수량', ascending=False).head(5)

        ranking_text = f"**[{target_team}] 품목 판매 랭킹 (Top 5)**\n"
        for _, row in top_items.iterrows():
            ranking_text += f"- {row['품명']} ({row['수량']}개)\n"

        # 벡터 DB에서 타겟 부서의 임베딩 가져오기
        target_embedding_result = team_collection.get(ids=[target_team], include=['embeddings'])
        if not target_embedding_result['embeddings']:
            return ranking_text, "해당 부서의 임베딩이 존재하지 않습니다."

        target_embedding = np.array(target_embedding_result['embeddings'][0])

        # 유사한 부서 찾기 (가장 가까운 3개 부서, 자신 포함)
        similar_teams_result = team_collection.query(
            query_embeddings=[target_embedding.tolist()],
            n_results=4,
            include=['documents', 'embeddings']
        )

        # 문서(원본 텍스트)와 임베딩 정보 추출
        results = similar_teams_result['documents'][0]
        result_embeddings = similar_teams_result['embeddings'][0]

        # --- 내적 유사도 계산 및 정렬 ---
        dot_product_scores = calculate_dot_product_similarity(target_embedding.tolist(), result_embeddings)

        scored_results = sorted(zip(dot_product_scores, results), key=lambda x: x[0], reverse=True)

        similar_teams = [doc for score, doc in scored_results if doc != target_team][:3]

        # --- 수정 끝 ---

        if not similar_teams:
            similar_text = "유사 부서를 찾을 수 없습니다."
        else:
            similar_teams_df = orders_df[orders_df['주문자소속팀'].isin(similar_teams)]

            if similar_teams_df.empty:
                similar_text = f"유사 부서: {', '.join(similar_teams)}\n\n유사 부서들의 판매 데이터가 충분하지 않습니다."
            else:
                similar_top_items = similar_teams_df.groupby('품목코드')['수량'].sum().reset_index()
                similar_top_items['품명'] = similar_top_items['품목코드'].map(item_code_to_name)
                similar_top_items = similar_top_items.sort_values(by='수량', ascending=False).head(5)

                similar_text = f"**[{target_team}]과 유사한 부서들의 선호 품목 (Top 5)**\n"
                similar_text += f"(유사 부서: {', '.join(similar_teams)})\n"
                for _, row in similar_top_items.iterrows():
                    similar_text += f"- {row['품명']} ({row['수량']}개)\n"
    except Exception as e:
        ranking_text = "오류 발생"
        similar_text = f"유사 부서 분석 중 오류 발생:\n{traceback.format_exc()}"

    return ranking_text, similar_text

# Gradio UI 구현
teams = sorted(orders_df['주문자소속팀'].unique().tolist())

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# PoC #1: 부서별 품목 선호도 분석")
    gr.Markdown("특정 부서를 선택하면, 해당 부서의 판매 랭킹과 AI가 추천하는 유사 부서의 선호 품목을 분석합니다.")

    with gr.Row():
        team_dropdown = gr.Dropdown(choices=teams, label="분석할 부서를 선택하세요", value=teams[0])
        run_button = gr.Button("분석 실행", variant="primary")

    with gr.Row():
        ranking_output = gr.Textbox(label="선택한 부서의 품목 판매 랭킹", lines=6)
        similar_output = gr.Textbox(label="유사 부서들의 선호 품목", lines=6)

    run_button.click(
        fn=analyze_team_preferences,
        inputs=team_dropdown,
        outputs=[ranking_output, similar_output]
    )

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

여기서 부터 다시 해보자

In [ ]:
# 1. Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 2. 필수 라이브러리 설치
!pip install pandas chromadb sentence-transformers gradio scikit-learn -q

In [ ]:
import pandas as pd
import chromadb
import os

# 파일 경로 설정
DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
DATA_PATH = os.path.join(DRIVE_PATH, 'dummy_data')
DB_PATH = os.path.join(DRIVE_PATH, 'db_data')

# 데이터 불러오기
orders_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_orders.csv'))
items_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_items.csv'))
item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()

# 지속성 모드로 ChromaDB 클라이언트 연결
client = chromadb.PersistentClient(path=DB_PATH)

# ==================== 수정된 부분 ====================
# .get_collection -> .get_or_create_collection 으로 변경
team_collection = client.get_or_create_collection("team_embeddings")
# ====================================================

print("✅ 데이터 로드 및 벡터 DB 연결 완료!")
print(f"팀 컬렉션의 데이터 수: {team_collection.count()}개")

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import gradio as gr
import numpy as np
import os
import traceback

# ======================================================================
# 1. 환경 설정: Google Drive 연결 및 라이브러리 설치
# ======================================================================
print("1. 환경 설정을 시작합니다...")
from google.colab import drive
drive.mount('/content/drive')
!pip install pandas chromadb sentence-transformers gradio -q
print("✅ 라이브러리 설치 완료!")

# ======================================================================
# 2. 데이터 및 경로 설정
# ======================================================================
print("\n2. 데이터 및 경로를 설정합니다...")
# 파일 경로 설정
DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
DATA_PATH = os.path.join(DRIVE_PATH, 'dummy_data')
DB_PATH = os.path.join(DRIVE_PATH, 'db_data')

# 데이터 불러오기
orders_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_orders.csv'))
items_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_items.csv'))
print("✅ 데이터 로드 완료!")

# ======================================================================
# 3. AI 임베딩 생성 및 영구 벡터 DB 구축
# ======================================================================
print("\n3. AI 임베딩 생성 및 영구 벡터 DB 구축을 시작합니다...")
# ChromaDB 클라이언트를 지속성 모드로 설정
client = chromadb.PersistentClient(path=DB_PATH)

# 기존 컬렉션이 있다면 삭제하여 초기화 (최초 실행 시 안전 장치)
try:
    client.delete_collection(name="item_embeddings")
    client.delete_collection(name="team_embeddings")
    print("기존 컬렉션을 초기화했습니다.")
except Exception:
    pass

# 새로운 컬렉션 생성
item_collection = client.get_or_create_collection("item_embeddings")
team_collection = client.get_or_create_collection("team_embeddings")

# 한국어 임베딩 모델 로드
print("임베딩 모델을 로드합니다... (시간이 소요될 수 있습니다)")
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 품목 임베딩 생성
print("품목 임베딩을 생성합니다...")
items_df['embedding_text'] = items_df['품명'] + ' ' + items_df['규격'] + ' ' + items_df['모델명'] + ' ' + items_df['제조원'] + ' ' + items_df['상품설명']
item_embeddings = model.encode(items_df['embedding_text'].tolist(), show_progress_bar=True)

# 부서 임베딩 생성
print("부서 임베딩을 생성합니다...")
teams = orders_df['주문자소속팀'].unique().tolist()
team_embeddings = model.encode(teams)

# 품목 임베딩 저장
print("생성된 품목 임베딩을 영구 DB에 저장합니다...")
item_collection.add(
    embeddings=item_embeddings.tolist(),
    metadatas=[{"품목코드": code} for code in items_df['품목코드'].tolist()],
    documents=items_df['embedding_text'].tolist(),
    ids=items_df['품목코드'].tolist()
)

# 부서 임베딩 저장
print("생성된 부서 임베딩을 영구 DB에 저장합니다...")
team_collection.add(
    embeddings=team_embeddings.tolist(),
    documents=teams,
    ids=teams
)

print("\n✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")
print("✅ 영구 벡터 DB 생성 및 데이터 저장 완료!")
print(f"품목 데이터 수: {item_collection.count()}개")
print(f"부서 데이터 수: {team_collection.count()}개")
print("✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")

여기서 테스트를 위해 런타임 종료



In [ ]:
# ======================================================================
# 1단계: 환경 설정 (Drive 연결 및 라이브러리 설치)
# ======================================================================
from google.colab import drive
drive.mount('/content/drive')
!pip install pandas chromadb sentence-transformers gradio -q
print("✅ 1단계: 환경 설정 완료!")

# ======================================================================
# 2단계: 데이터 로드 및 영구 벡터 DB 연결
# ======================================================================
import pandas as pd
import chromadb
import os

DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
DATA_PATH = os.path.join(DRIVE_PATH, 'dummy_data')
DB_PATH = os.path.join(DRIVE_PATH, 'db_data')

orders_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_orders.csv'))
items_df = pd.read_csv(os.path.join(DATA_PATH, 'dummy_items.csv'))
item_code_to_name = pd.Series(items_df.품명.values, index=items_df.품목코드).to_dict()

client = chromadb.PersistentClient(path=DB_PATH)
team_collection = client.get_collection("team_embeddings") # 이제 get_or_create가 아닌 get을 사용해도 됩니다.

print(f"✅ 2단계: 데이터 로드 및 벡터 DB 연결 완료! (팀 데이터 수: {team_collection.count()}개)")


# ======================================================================
# 3단계: 분석 함수 및 Gradio UI 실행 (수정 완료)
# ======================================================================
import gradio as gr
import numpy as np
import traceback

def calculate_dot_product_similarity(query_embedding: list, result_embeddings: list):
    query_vector = np.array(query_embedding)
    result_vectors = np.array(result_embeddings)
    dot_product_scores = np.dot(result_vectors, query_vector)
    return dot_product_scores.tolist()

def analyze_team_preferences(target_team: str):
    try:
        team_sales = orders_df[orders_df['주문자소속팀'] == target_team]
        if team_sales.empty: return f"[{target_team}]의 판매 데이터가 없습니다.", "분석할 유사 부서를 찾을 수 없습니다."
        top_items = team_sales.groupby('품목코드')['수량'].sum().reset_index()
        top_items['품명'] = top_items['품목코드'].map(item_code_to_name)
        top_items = top_items.sort_values(by='수량', ascending=False).head(5)
        ranking_text = f"**[{target_team}] 품목 판매 랭킹 (Top 5)**\n"
        for _, row in top_items.iterrows(): ranking_text += f"- {row['품명']} ({row['수량']}개)\n"

        target_embedding_result = team_collection.get(ids=[target_team], include=['embeddings'])

        # ==================== 수정된 부분 ====================
        if len(target_embedding_result['embeddings']) == 0:
        # ====================================================
            return ranking_text, "해당 부서의 임베딩이 존재하지 않습니다."

        target_embedding = np.array(target_embedding_result['embeddings'][0])
        similar_teams_result = team_collection.query(query_embeddings=[target_embedding.tolist()], n_results=4, include=['documents', 'embeddings'])
        results, result_embeddings = similar_teams_result['documents'][0], similar_teams_result['embeddings'][0]
        dot_product_scores = calculate_dot_product_similarity(target_embedding.tolist(), result_embeddings)
        scored_results = sorted(zip(dot_product_scores, results), key=lambda x: x[0], reverse=True)
        similar_teams = [doc for score, doc in scored_results if doc != target_team][:3]
        if not similar_teams: similar_text = "유사 부서를 찾을 수 없습니다."
        else:
            similar_teams_df = orders_df[orders_df['주문자소속팀'].isin(similar_teams)]
            if similar_teams_df.empty: similar_text = f"유사 부서: {', '.join(similar_teams)}\n\n유사 부서들의 판매 데이터가 충분하지 않습니다."
            else:
                similar_top_items = similar_teams_df.groupby('품목코드')['수량'].sum().reset_index()
                similar_top_items['품명'] = similar_top_items['품목코드'].map(item_code_to_name)
                similar_top_items = similar_top_items.sort_values(by='수량', ascending=False).head(5)
                similar_text = f"**[{target_team}]과 유사한 부서들의 선호 품목 (Top 5)**\n"
                similar_text += f"(유사 부서: {', '.join(similar_teams)})\n"
                for _, row in similar_top_items.iterrows(): similar_text += f"- {row['품명']} ({row['수량']}개)\n"
    except Exception as e:
        ranking_text, similar_text = "오류 발생", f"분석 중 오류 발생:\n{traceback.format_exc()}"
    return ranking_text, similar_text

teams = sorted(orders_df['주문자소속팀'].unique().tolist())
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# PoC #1: 부서별 품목 선호도 분석")
    gr.Markdown("특정 부서를 선택하면, 해당 부서의 판매 랭킹과 AI가 추천하는 유사 부서의 선호 품목을 분석합니다.")
    with gr.Row():
        team_dropdown = gr.Dropdown(choices=teams, label="분석할 부서를 선택하세요", value=teams[0])
        run_button = gr.Button("분석 실행", variant="primary")
    with gr.Row():
        ranking_output = gr.Textbox(label="선택한 부서의 품목 판매 랭킹", lines=6)
        similar_output = gr.Textbox(label="유사 부서들의 선호 품목", lines=6)
    run_button.click(fn=analyze_team_preferences, inputs=team_dropdown, outputs=[ranking_output, similar_output])

print("✅ 3단계: Gradio UI를 실행합니다...")
demo.launch(share=True, debug=True)

1.   잠정적 POC # 1 작업 완료 (주문 데이터 생성 필요)

POC #2
1. 품목 정보 백터 DB 화
2. 네이버 상품 API 연동을 통한 품목 정비
3. 유사 상품 추천 로직 개발
4. UI 구성

In [ ]:
# 필수 라이브러리 설치
!pip install pandas chromadb sentence-transformers -q

print("✅ 라이브러리 설치 완료!")

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os

# ======================================================================
# 0. 환경 설정 및 데이터 로드
# ======================================================================
# 이전 단계에서 API 정보로 보강이 완료된 데이터 파일 경로
file_path = '/content/drive/MyDrive/ai_poc_project/original_data/item_data_enriched.csv'

try:
    # Google Drive 연결
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # 라이브러리 설치
    !pip install pandas chromadb sentence-transformers -q

    enriched_df = pd.read_csv(file_path)
    print(f"✅ 총 {len(enriched_df)}개 보강된 품목 데이터 로드 완료.")
except FileNotFoundError:
    print(f"❌ [에러] '{file_path}' 파일을 찾을 수 없습니다.")
    print("이전 단계의 데이터 보강 코드가 성공적으로 실행되었는지 확인해주세요.")
except Exception as e:
    print(f"❌ [에러] 파일을 읽는 중 오류가 발생했습니다: {e}")


# ======================================================================
# 1. "AI 학습 자료" (embedding_text) 최종 재구성
# ======================================================================
print("\n1. 모든 정보를 반영하여 AI 학습 자료를 최종 구성합니다...")

# 학습에 사용할 전체 컬럼 리스트
cols_to_embed = [
    '품목코드', '한글품명', '영문품명', '규격', '모델명', '제조원명',
    '네이버_카테고리1', '네이버_카테고리2', '네이버_카테고리3', '네이버_카테고리4',
    '브랜드', '최저가'
]

# 빈 값(NaN)을 안전하게 처리하고, 모든 컬럼을 문자열로 변환하여 결합
enriched_df['embedding_text'] = enriched_df[cols_to_embed].fillna('').astype(str).agg(' '.join, axis=1)

print("✅ 최종 embedding_text 생성 완료!")
print("\n[생성된 embedding_text 샘플]")
display(enriched_df[['품목코드', 'embedding_text']].head())


# ======================================================================
# 2. 최종 벡터 생성 및 영구 DB 업데이트
# ======================================================================
print("\n2. 최종 벡터 생성 및 영구 DB 업데이트를 시작합니다...")

# 영구 DB 경로 설정 및 클라이언트 연결
DB_PATH = '/content/drive/MyDrive/ai_poc_project/db_data'
client = chromadb.PersistentClient(path=DB_PATH)

# AI 임베딩 모델 로드
model_name = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
print(f"   - AI 임베딩 모델({model_name})을 로드합니다...")
model = SentenceTransformer(model_name)

# 최종 품목별 임베딩 생성
print("   - 모든 정보가 포함된 벡터를 생성합니다... (시간이 소요될 수 있습니다)")
item_embeddings = model.encode(enriched_df['embedding_text'].tolist(), show_progress_bar=True)

# 기존 컬렉션 삭제 (초기화)
try:
    client.delete_collection(name="item_embeddings")
    print("   - 기존 item_embeddings 컬렉션을 성공적으로 삭제했습니다.")
except Exception:
    pass

# 새로운 컬렉션 생성 및 최종 데이터 저장
print("   - 최종 벡터를 영구 DB에 저장합니다...")
item_collection = client.get_or_create_collection("item_embeddings")
item_collection.add(
    embeddings=item_embeddings.tolist(),
    metadatas=[{"품목코드": code} for code in enriched_df['품목코드'].tolist()],
    documents=enriched_df['embedding_text'].tolist(),
    ids=[str(code) for code in enriched_df['품목코드'].tolist()]
)

print("\n✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")
print("✅ [성공] 모든 정보가 반영된 최종 벡터 DB 구축 완료!")
print(f"✅ 최종 저장된 품목 데이터 수: {item_collection.count()}개")
print("✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")

In [ ]:
import pandas as pd
import requests
import os
import time
from tqdm import tqdm

# --- 0. 환경 설정 및 데이터 로드 ---
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass
!pip install tqdm -q
file_path = '/content/drive/MyDrive/ai_poc_project/original_data/item_data.txt'
real_items_df = pd.read_csv(file_path, sep='\t')

# --- 1. Naver API 설정 ---
CLIENT_ID = "1uypXVdrAyuahn7jp05g"
CLIENT_SECRET = "zzWqd49IJv"
headers = {"X-Naver-Client-Id": CLIENT_ID, "X-Naver-Client-Secret": CLIENT_SECRET}

# --- 2. API 호출 및 데이터 보강 (수정된 로직) ---
print("\nNaver API를 통해 데이터 보강을 시작합니다...")
api_results = []
for index, row in tqdm(real_items_df.iterrows(), total=real_items_df.shape[0]):
    query = row['한글품명']
    url = f"https://openapi.naver.com/v1/search/shop.json?query={query}&display=1"

    result = { # 모든 키를 미리 정의하여 KeyError 방지
        '네이버_카테고리1': 'N/A', '네이버_카테고리2': 'N/A', '네이버_카테고리3': 'N/A',
        '네이버_카테고리4': 'N/A', '브랜드': 'N/A', '이미지_URL': 'N/A',
        '최저가': 0, '상품_링크': 'N/A'
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200 and response.json()['items']:
            item = response.json()['items'][0]
            result.update({
                '네이버_카테고리1': item.get('category1', 'N/A'),
                '네이버_카테고리2': item.get('category2', 'N/A'),
                '네이버_카테고리3': item.get('category3', 'N/A'),
                '네이버_카테고리4': item.get('category4', 'N/A'),
                '브랜드': item.get('brand', 'N/A'),
                '이미지_URL': item.get('image', 'N/A'),
                '최저가': int(item.get('lprice', 0)),
                '상품_링크': item.get('link', 'N/A')
            })
    except Exception: pass
    api_results.append(result)
    time.sleep(0.05)

enriched_df = pd.concat([real_items_df.reset_index(drop=True), pd.DataFrame(api_results)], axis=1)
output_path = '/content/drive/MyDrive/ai_poc_project/original_data/item_data_enriched.csv'
enriched_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ 보강된 데이터가 정상적으로 저장되었습니다:\n{output_path}")

In [ ]:
위에 품목 정비는 월요일에 한번 더 수행하자

아래 소스는 POC1,2차 통합 소스 . 2차 유사 상품은 실시간 검색이고, 유사도는 0.75 이상만 추출 한다.

지속성을 남기는 방식에서 크롬 DB 와 클라우드 환경 간 이슈 발생 하여 방향성을 바꿔보자.

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os
import shutil

# --- 0. 환경 설정 및 데이터 로드 ---
print("환경 설정 및 데이터 로드를 시작합니다...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass
!pip install pandas chromadb sentence-transformers -q

file_path = '/content/drive/MyDrive/ai_poc_project/original_data/item_data_enriched.csv'
try:
    enriched_df = pd.read_csv(file_path)
    print(f"✅ 총 {len(enriched_df)}개 보강된 품목 데이터 로드 완료.")
except Exception as e:
    print(f"❌ [에러] 파일을 읽는 중 오류가 발생했습니다: {e}")
    raise SystemExit("데이터 로드 실패.")

# --- 1. "AI 학습 자료" (embedding_text) 최종 재구성 ---
print("\n1. 모든 정보를 반영하여 AI 학습 자료를 최종 구성합니다...")
cols_to_embed = [
    '품목코드', '한글품명', '영문품명', '규격', '모델명', '제조원명',
    '네이버_카테고리1', '네이버_카테고리2', '네이버_카테고리3', '네이버_카테고리4',
    '브랜드', '최저가'
]
enriched_df['embedding_text'] = enriched_df[cols_to_embed].fillna('').astype(str).agg(' '.join, axis=1)
print("✅ 최종 embedding_text 생성 완료!")

# --- 2. 최종 벡터 생성 및 "로컬 DB" 업데이트 ---
print("\n2. 최종 벡터 생성 및 '로컬 DB' 업데이트를 시작합니다...")
# ======== 저장 경로 변경 ========
DB_PATH = '/content/db_data' # Google Drive가 아닌 Colab 로컬 경로
# ============================

if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH)
os.makedirs(DB_PATH, exist_ok=True)

client = chromadb.PersistentClient(path=DB_PATH)
model_name = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
print(f"   - AI 임베딩 모델({model_name})을 로드합니다...")
model = SentenceTransformer(model_name)

print("   - 모든 정보가 포함된 벡터를 생성합니다... (시간이 소요될 수 있습니다)")
item_embeddings = model.encode(enriched_df['embedding_text'].tolist(), show_progress_bar=True)

item_collection = client.get_or_create_collection("item_embeddings")
item_collection.add(
    embeddings=item_embeddings.tolist(),
    metadatas=[{"품목코드": code} for code in enriched_df['품목코드'].tolist()],
    documents=enriched_df['embedding_text'].tolist(),
    ids=[str(code) for code in enriched_df['품목코드'].tolist()]
)

print("\n✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")
print(f"✅ [성공] 로컬 벡터 DB 재구축 완료! (경로: {DB_PATH})")
print(f"✅ 최종 저장된 품목 데이터 수: {item_collection.count()}개")
print("✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")

In [ ]:
import shutil
import os
import pandas as pd

# --- 경로 설정 ---
local_db_path = '/content/db_data'
backup_drive_folder = '/content/drive/MyDrive/ai_poc_project/db_backup'
# --- 경로 설정 끝 ---

print(f"'{local_db_path}' 경로의 로컬 DB를 Google Drive에 백업합니다...")

try:
    os.makedirs(backup_drive_folder, exist_ok=True)
    backup_filename = f"db_backup_{pd.Timestamp.now().strftime('%Y%m%d')}"

    # 압축할 파일의 전체 경로
    zip_file_path_local = f"{backup_filename}.zip"
    zip_file_path_drive = os.path.join(backup_drive_folder, zip_file_path_local)

    # 1. 로컬 DB 폴더를 zip 파일로 압축
    shutil.make_archive(backup_filename, 'zip', local_db_path)
    print(f"   - 로컬 DB를 '{zip_file_path_local}' 파일로 압축했습니다.")

    # ======== 수정된 부분 ========
    # 2. 만약 Google Drive에 동일한 이름의 파일이 이미 있다면, 먼저 삭제
    if os.path.exists(zip_file_path_drive):
        os.remove(zip_file_path_drive)
        print(f"   - 기존 백업 파일({zip_file_path_drive})을 삭제했습니다.")
    # ============================

    # 3. 새로 생성된 zip 파일을 Google Drive 백업 폴더로 이동
    shutil.move(zip_file_path_local, backup_drive_folder)

    print(f"\n✅ [성공] 로컬 DB를 Google Drive에 백업 완료!")
    print(f"   - 백업 파일 위치: {zip_file_path_drive}")

except Exception as e:
    print(f"\n❌ [에러] 백업 중 오류가 발생했습니다: {e}")

In [ ]:
import pandas as pd
import chromadb
import gradio as gr
import numpy as np
import os
import shutil
import traceback
from IPython.display import display, HTML
import stat # 파일 권한 변경을 위한 라이브러리

# ======================================================================
# 1. 환경 설정 및 DB 복원 (권한 문제 해결)
# ======================================================================
print("1. 환경 설정 및 DB 복원을 시작합니다...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass
!pip install pandas chromadb sentence-transformers gradio -q

backup_zip_path = '/content/drive/MyDrive/ai_poc_project/db_backup/db_backup_20250929.zip'
local_db_path = '/content/db_data'

try:
    if os.path.exists(local_db_path):
        shutil.rmtree(local_db_path)
    shutil.unpack_archive(backup_zip_path, local_db_path, 'zip')
    print(f"✅ [성공] DB 복원 완료! (경로: {local_db_path})")

    # ======== 수정된 부분 ========
    # 복원된 모든 파일 및 폴더에 쓰기 권한(chmod 755) 부여
    print("   - 복원된 DB 파일에 쓰기 권한을 부여합니다...")
    for root, dirs, files in os.walk(local_db_path):
        for d in dirs:
            os.chmod(os.path.join(root, d), stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)
        for f in files:
            os.chmod(os.path.join(root, f), stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH)
    print("   - 권한 부여 완료.")
    # ============================

except Exception as e:
    print(f"❌ [에러] DB 복원/권한 설정 중 오류가 발생했습니다: {e}")
    raise SystemExit("DB 복원 실패.")

# ======================================================================
# 2. 분석 함수 정의
# ======================================================================

def calculate_cosine_similarity(query_embedding, result_embeddings):
    query_vector = np.array(query_embedding)
    result_vectors = np.array(result_embeddings)
    query_norm = np.linalg.norm(query_vector)
    if query_norm == 0: return [0.0] * len(result_vectors)
    query_vector /= query_norm
    result_norms = np.linalg.norm(result_vectors, axis=1, keepdims=True)
    result_vectors /= np.where(result_norms == 0, 1, result_norms)
    return np.dot(result_vectors, query_vector).tolist()

### PoC #2: 품목별 유사 상품 추천 함수 ###
def analyze_item_similarity(target_item_identifier: str):
    try:
        if not target_item_identifier:
            return "분석할 품목을 선택 또는 입력해주세요.", ""

        if ':' in target_item_identifier:
            target_item_code = str(target_item_identifier.split(':')[0].strip())
        else:
            target_item_code = str(target_item_identifier.strip())

        selected_item_series = enriched_df[enriched_df['품목코드'].astype(str) == target_item_code].iloc[0]

        target_embedding = item_collection.get(ids=[target_item_code], include=['embeddings'])['embeddings'][0]
        similar_items_result = item_collection.query(query_embeddings=[target_embedding], n_results=15, include=['metadatas', 'embeddings'])

        result_embeddings = similar_items_result['embeddings'][0]
        result_metadatas = similar_items_result['metadatas'][0]
        similarity_scores = calculate_cosine_similarity(target_embedding, result_embeddings)

        qualified_items = []
        for score, meta in zip(similarity_scores, result_metadatas):
            if str(meta['품목코드']) != target_item_code and score >= 0.75:
                item_details = enriched_df[enriched_df['품목코드'] == meta['품목코드']].iloc[0].to_dict()
                item_details['유사도'] = score
                qualified_items.append(item_details)

        recommended_df = pd.DataFrame(qualified_items).sort_values(by='유사도', ascending=False).head(5)

        selected_item_html = f"""
        <h4>선택된 품목 정보</h4>
        <div style="display:flex; align-items:center; border: 1px solid #E0E0E0; padding: 10px; border-radius: 5px;">
            <img src="{selected_item_series.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;">
            <div>
                <b>{selected_item_series.get('한글품명', '')} ({selected_item_series.get('품목코드', '')})</b><br>
                <b>카테고리:</b> {selected_item_series.get('네이버_카테고리1', '')} > {selected_item_series.get('네이버_카테고리2', '')}<br>
                <b>규격:</b> {selected_item_series.get('규격', '')}
            </div>
        </div>
        """

        recommend_html = "<h4>AI 추천 유사 품목</h4>"
        if recommended_df.empty:
            recommend_html += "<p>기준점수(0.75)를 넘는 유사 품목을 찾을 수 없습니다.</p>"
        else:
            for _, row in recommended_df.iterrows():
                recommend_html += f"""
                <div style="display:flex; align-items:center; border: 1px solid #4CAF50; padding: 10px; border-radius: 5px; margin-bottom:10px;">
                    <img src="{row.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;">
                    <div>
                        <b>{row.get('한글품명', '')} ({row.get('품목코드', '')}) - <span style="color:#4CAF50; font-weight:bold;">유사도: {row.get('유사도', 0):.2f}</span></b><br>
                        <b>카테고리:</b> {row.get('네이버_카테고리1', '')} > {row.get('네이버_카테고리2', '')}<br>
                        <b>규격:</b> {row.get('규격', '')}
                    </div>
                </div>
                """
        return selected_item_html, recommend_html

    except IndexError:
        return f"<h4>오류</h4><p>입력하신 품목코드 '{target_item_code}'를 찾을 수 없습니다.</p>", ""
    except Exception as e:
        return f"<h4>오류 발생</h4><p>{traceback.format_exc()}</p>", ""

### PoC #1: 유사 부서 상품 추천 함수 ###
def analyze_team_preferences(target_team: str):
    try:
        team_sales = orders_df[orders_df['주문자소속팀'] == target_team]
        if team_sales.empty:
            return f"[{target_team}]의 판매 데이터가 없습니다.", "분석할 유사 부서를 찾을 수 없습니다."

        top_items = team_sales.groupby('품목코드')['수량'].sum().reset_index()
        top_items['품명'] = top_items['품목코드'].map(item_code_to_name_poc1)
        top_items = top_items.sort_values(by='수량', ascending=False).head(5)

        ranking_text = f"**[{target_team}] 품목 판매 랭킹 (Top 5)**\n"
        for _, row in top_items.iterrows():
            ranking_text += f"- {row['품명']} ({row['수량']}개)\n"

        target_embedding_result = team_collection.get(ids=[target_team], include=['embeddings'])
        if len(target_embedding_result['embeddings']) == 0:
            return ranking_text, "해당 부서의 임베딩이 존재하지 않습니다."

        target_embedding = np.array(target_embedding_result['embeddings'][0])
        similar_teams_result = team_collection.query(query_embeddings=[target_embedding.tolist()], n_results=15, include=['documents', 'embeddings'])

        results = similar_teams_result['documents'][0]
        result_embeddings = similar_teams_result['embeddings'][0]

        similarity_scores = calculate_cosine_similarity(target_embedding.tolist(), result_embeddings)
        scored_results = sorted(zip(similarity_scores, results), key=lambda x: x[0], reverse=True)
        similar_teams = [doc for score, doc in scored_results if doc != target_team][:3]

        if not similar_teams:
            similar_text = "유사 부서를 찾을 수 없습니다."
        else:
            similar_teams_df = orders_df[orders_df['주문자소속팀'].isin(similar_teams)]
            if similar_teams_df.empty:
                similar_text = f"유사 부서: {', '.join(similar_teams)}\n\n유사 부서들의 판매 데이터가 충분하지 않습니다."
            else:
                similar_top_items = similar_teams_df.groupby('품목코드')['수량'].sum().reset_index()
                similar_top_items['품명'] = similar_top_items['품목코드'].map(item_code_to_name_poc1)
                similar_top_items = similar_top_items.sort_values(by='수량', ascending=False).head(5)
                similar_text = f"**[{target_team}]과 유사한 부서들의 선호 품목 (Top 5)**\n"
                similar_text += f"(유사 부서: {', '.join(similar_teams)})\n"
                for _, row in similar_top_items.iterrows():
                    similar_text += f"- {row['품명']} ({row['수량']}개)\n"
    except Exception as e:
        ranking_text, similar_text = "오류 발생", f"분석 중 오류 발생:\n{traceback.format_exc()}"
    return ranking_text, similar_text


# ======================================================================
# 3. Gradio UI 통합 구현 (v3.1)
# ======================================================================
print("\n3. Gradio UI (v3.1)를 실행합니다...")

teams_list = sorted(orders_df['주문자소속팀'].unique().tolist())
categories_list = sorted(enriched_df['네이버_카테고리1'].dropna().unique().tolist())
enriched_df['item_display'] = enriched_df['품목코드'].astype(str) + " : " + enriched_df['한글품명']

def update_item_dropdown(category):
    if not category:
        return gr.Dropdown(choices=[], label="2. 품목을 선택하세요")
    filtered_df = enriched_df[enriched_df['네이버_카테고리1'] == category]
    choices = sorted(filtered_df['item_display'].dropna().unique().tolist())
    return gr.Dropdown(choices=choices, label="2. 품목을 선택하세요", value=choices[0] if choices else None)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 IMK Plus AI PoC Project")

    with gr.Tabs():
        # ---------- PoC #1 UI ----------
        with gr.TabItem("PoC #1: 유사 부서 상품 추천"):
            gr.Markdown("특정 부서를 선택하면, 해당 부서의 판매 랭킹과 AI가 추천하는 유사 부서의 선호 품목을 분석합니다.")
            with gr.Row():
                team_dropdown = gr.Dropdown(choices=teams_list, label="분석할 부서를 선택하세요")
                poc1_button = gr.Button("분석 실행", variant="primary")
            with gr.Row():
                ranking_output = gr.Textbox(label="선택한 부서의 품목 판매 랭킹", lines=7)
                similar_output = gr.Textbox(label="유사 부서들의 선호 품목", lines=7)
            poc1_button.click(fn=analyze_team_preferences, inputs=team_dropdown, outputs=[ranking_output, similar_output])

        # ---------- PoC #2 UI ----------
        with gr.TabItem("PoC #2: 품목별 유사 상품 추천"):
            gr.Markdown("AI가 **유사도 0.75 이상**인 품목 중 가장 유사한 **최대 5개**를 추천합니다.")

            with gr.Tabs():
                with gr.TabItem("카테고리로 찾기"):
                    with gr.Row():
                        category_dd = gr.Dropdown(choices=categories_list, label="1. 대분류 카테고리를 선택하세요")
                        item_dd = gr.Dropdown(label="2. 품목을 선택하세요 (카테고리 먼저 선택)", interactive=True)
                    cat_button = gr.Button("유사 품목 찾기", variant="primary")

                with gr.TabItem("직접 검색하기"):
                    item_code_tb = gr.Textbox(label="품목코드를 직접 입력하세요", placeholder="예: 1419128400")
                    code_button = gr.Button("유사 품목 찾기", variant="primary")

            gr.HTML("<hr style='margin: 20px 0;'>")
            selected_item_output = gr.HTML()
            similar_items_output = gr.HTML()

            category_dd.change(fn=update_item_dropdown, inputs=category_dd, outputs=item_dd)
            cat_button.click(fn=analyze_item_similarity, inputs=item_dd, outputs=[selected_item_output, similar_items_output])
            code_button.click(fn=analyze_item_similarity, inputs=item_code_tb, outputs=[selected_item_output, similar_items_output])

demo.launch(share=True, debug=True)

부서정보 없어서 에러 발생 일단 주문정보/딜리버리 정보 업로드 하자


In [ ]:
import pandas as pd
import os
import traceback

# ======================================================================
# 0. 환경 설정 및 데이터 로드
# ======================================================================
print("환경 설정 및 데이터 로드를 시작합니다...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass

# --- 파일 경로 설정 ---
DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
ORIGINAL_DATA_PATH = os.path.join(DRIVE_PATH, 'original_data')
ZIPCODE_DATA_PATH = os.path.join(DRIVE_PATH, 'zipcode_data')

# --- 데이터 로드 ---
try:
    delivery_df = pd.read_csv(os.path.join(ORIGINAL_DATA_PATH, 'delivery_data_cleaned.txt'), sep='\t')
    print(f"✅ 배송 데이터 로드 완료: {len(delivery_df)}건")

    zipcode_db_df = pd.read_csv(os.path.join(ZIPCODE_DATA_PATH, 'all_zipcodes.csv'))
    print(f"✅ 우편번호 DB 로드 완료: {len(zipcode_db_df)}건")

    # ======== 수정된 부분 ========
    # 두 데이터프레임의 '우편번호' 컬럼을 모두 '문자열' 타입으로 통일
    delivery_df['우편번호'] = delivery_df['우편번호'].astype(str)
    zipcode_db_df['우편번호'] = zipcode_db_df['우편번호'].astype(str)
    print("   - '우편번호' 컬럼 타입 통일 완료.")
    # ============================

except Exception as e:
    print(f"❌ 데이터 로드 중 오류가 발생했습니다: {e}")
    raise SystemExit("데이터 로드 실패.")

# ======================================================================
# 1. 데이터 전처리
# ======================================================================
print("\n1. 데이터 전처리를 시작합니다...")

try:
    # --- 1-1. '리드 타임' 계산 ---
    delivery_df['운송지시일자'] = pd.to_datetime(delivery_df['운송지시일자'], format='%Y%m%d', errors='coerce')
    delivery_df['운송완료일자'] = pd.to_datetime(delivery_df['운송완료일자'], format='%Y%m%d', errors='coerce')
    delivery_df['리드타임'] = (delivery_df['운송완료일자'] - delivery_df['운송지시일자']).dt.days
    print("   - '리드타임' 컬럼 생성 완료.")

    # --- 1-2. '도시' 정보 통합 매핑 ---
    zipcode_map = zipcode_db_df[['우편번호', '시도']].rename(columns={'시도': '도시'})
    preprocessed_df = pd.merge(delivery_df, zipcode_map, on='우편번호', how='left')
    print("   - '우편번호'를 기준으로 '도시' 정보 전체 매핑 완료.")

    # --- 1-3. 컬럼명 변경 및 최종 정제 ---
    preprocessed_df.rename(columns={'납품물류유형': '배송유형'}, inplace=True)
    final_cols = ['품목코드', '배송유형', '도시', '리드타임']
    final_df = preprocessed_df[final_cols].copy()

    final_df.dropna(inplace=True)
    final_df = final_df[final_df['리드타임'] >= 0]
    print("   - 최종 데이터 정제 완료.")

    # ======================================================================
    # 2. 결과 확인
    # ======================================================================
    print("\n✨ [성공] AI 학습용 데이터 준비가 완료되었습니다.")
    print(f"   - 최종 분석용 데이터: {len(final_df)}건")
    print("\n[최종 데이터 샘플 (상위 5개)]")
    display(final_df.head())

    print("\n[도시별 데이터 건수]")
    print(final_df['도시'].value_counts())

except Exception as e:
    print(f"\n❌ [에러] 전처리 중 오류가 발생했습니다: {e}")
    print(traceback.format_exc())

주문 데이터 적치

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import os
import shutil
import stat

# --- 0. 환경 설정 및 데이터 로드 ---
print("환경 설정 및 데이터 로드를 시작합니다...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass
!pip install pandas chromadb sentence-transformers -q

local_db_path = '/content/db_data'
file_path = '/content/drive/MyDrive/ai_poc_project/original_data/orders_final.txt'

try:
    # --- 1. DB 복원 ---
    backup_zip_path = f"/content/drive/MyDrive/ai_poc_project/db_backup/db_backup_{pd.Timestamp.now().strftime('%Y%m%d')}.zip"
    if os.path.exists(local_db_path): shutil.rmtree(local_db_path)
    shutil.unpack_archive(backup_zip_path, local_db_path, 'zip')
    print("✅ 기존 DB 복원 완료.")
    for root, dirs, files in os.walk(local_db_path):
        for d in dirs: os.chmod(os.path.join(root, d), stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)
        for f in files: os.chmod(os.path.join(root, f), stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH)

    # --- 2. 신규 주문 데이터 로드 ---
    orders_df = pd.read_csv(file_path, sep='\t')
    print(f"✅ 신규 주문 데이터 로드 완료: {len(orders_df)}건")

    # ======== 수정된 부분 (강화된 정제 로직) ========
    # --- 2-1. 주문 데이터 정제 ---
    # '주문수량'을 숫자로 변환 (숫자가 아니면 강제로 NaT/NaN 값으로 변경)
    orders_df['주문수량'] = pd.to_numeric(orders_df['주문수량'], errors='coerce')
    # '주문수량'에 유효한 숫자가 없는 모든 행(row) 제거
    orders_df.dropna(subset=['주문수량'], inplace=True)
    # '주문수량'을 정수(integer) 타입으로 최종 변환
    orders_df['주문수량'] = orders_df['주문수량'].astype(int)
    print(f"   - '주문수량' 정제 완료. (유효 데이터: {len(orders_df)}건)")
    # ===============================================

except Exception as e:
    print(f"❌ [에러] 초기 설정 중 오류가 발생했습니다: {e}")
    raise SystemExit("초기 설정 실패.")

# --- 3. 부서별 구매 패턴으로 "AI 학습 자료" 재구성 ---
print("\n부서별 구매 패턴을 분석하여 학습 자료를 구성합니다...")
team_purchase_list = orders_df.loc[orders_df.index.repeat(orders_df['주문수량'])]
team_embedding_text = team_purchase_list.groupby('주문자부서명')['품목코드'].apply(lambda x: ' '.join(x.astype(str)))
team_df = team_embedding_text.reset_index()
print("✅ 부서별 학습 자료 생성 완료.")

# --- 4. 부서(팀) 벡터 생성 및 "로컬 DB" 업데이트 ---
print("\n'team_embeddings' 컬렉션을 새로 구축합니다...")
client = chromadb.PersistentClient(path=local_db_path)
model_name = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
model = SentenceTransformer(model_name)

team_embeddings = model.encode(team_df['품목코드'].tolist(), show_progress_bar=True)

try:
    client.delete_collection(name="team_embeddings")
    print("   - 기존 team_embeddings 컬렉션을 삭제했습니다.")
except Exception:
    print("   - 신규 team_embeddings 컬렉션을 생성합니다.")

team_collection = client.get_or_create_collection("team_embeddings")
team_collection.add(
    embeddings=team_embeddings.tolist(),
    documents=team_df['주문자부서명'].tolist(),
    ids=team_df['주문자부서명'].tolist()
)

print("\n✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")
print(f"✅ [성공] 로컬 DB에 'team_embeddings' 재구축 완료!")
print(f"   - 최종 저장된 부서 데이터 수: {team_collection.count()}개")
print("✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")

In [ ]:
import shutil
import os
import pandas as pd

# --- 경로 설정 ---
local_db_path = '/content/db_data'
backup_drive_folder = '/content/drive/MyDrive/ai_poc_project/db_backup'
# --- 경로 설정 끝 ---

print(f"\n업데이트된 로컬 DB를 Google Drive에 백업합니다...")

try:
    os.makedirs(backup_drive_folder, exist_ok=True)
    backup_filename = f"db_backup_{pd.Timestamp.now().strftime('%Y%m%d')}"
    zip_file_path_local = f"{backup_filename}.zip"
    zip_file_path_drive = os.path.join(backup_drive_folder, zip_file_path_local)

    if os.path.exists(zip_file_path_drive):
        os.remove(zip_file_path_drive)
        print(f"   - 기존 백업 파일({zip_file_path_drive})을 삭제했습니다.")

    shutil.make_archive(backup_filename, 'zip', local_db_path)
    shutil.move(zip_file_path_local, backup_drive_folder)

    print(f"\n✅ [성공] 업데이트된 DB 백업 완료!")
    print(f"   - 백업 파일 위치: {zip_file_path_drive}")

except Exception as e:
    print(f"\n❌ [에러] 백업 중 오류가 발생했습니다: {e}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import os
import traceback

# ======================================================================
# 0. 데이터 전처리 (이전 단계 요약)
# ======================================================================
print("0. AI 학습용 데이터를 준비합니다...")
try:
    # Google Drive 연결
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # 경로 설정
    DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
    ORIGINAL_DATA_PATH = os.path.join(DRIVE_PATH, 'original_data')
    ZIPCODE_DATA_PATH = os.path.join(DRIVE_PATH, 'zipcode_data')

    # 데이터 로드
    delivery_df = pd.read_csv(os.path.join(ORIGINAL_DATA_PATH, 'delivery_data_cleaned.txt'), sep='\t')
    zipcode_db_df = pd.read_csv(os.path.join(ZIPCODE_DATA_PATH, 'all_zipcodes.csv'))

    # 타입 통일
    delivery_df['우편번호'] = delivery_df['우편번호'].astype(str)
    zipcode_db_df['우편번호'] = zipcode_db_df['우편번호'].astype(str)

    # 전처리
    delivery_df['운송지시일자'] = pd.to_datetime(delivery_df['운송지시일자'], format='%Y%m%d', errors='coerce')
    delivery_df['운송완료일자'] = pd.to_datetime(delivery_df['운송완료일자'], format='%Y%m%d', errors='coerce')
    delivery_df['리드타임'] = (delivery_df['운송완료일자'] - delivery_df['운송지시일자']).dt.days
    zipcode_map = zipcode_db_df[['우편번호', '시도']].rename(columns={'시도': '도시'})
    preprocessed_df = pd.merge(delivery_df, zipcode_map, on='우편번호', how='left')
    preprocessed_df.rename(columns={'납품물류유형': '배송유형'}, inplace=True)
    final_df = preprocessed_df[['품목코드', '배송유형', '도시', '리드타임']].copy()
    final_df.dropna(inplace=True)
    final_df = final_df[final_df['리드타임'] >= 0]
    print(f"✅ 데이터 준비 완료. (총 {len(final_df)}건)")

except Exception as e:
    print(f"❌ 데이터 준비 중 오류: {e}")
    raise SystemExit("실패")


# ======================================================================
# 1. AI 학습을 위한 데이터 변환 (One-Hot Encoding)
# ======================================================================
print("\n1. 전체 데이터를 AI가 학습할 수 있도록 변환합니다...")
features_df = pd.get_dummies(final_df, columns=['품목코드', '배송유형', '도시'])
X = features_df.drop('리드타임', axis=1)
y = features_df['리드타임']
print("✅ 데이터 변환 완료.")

# ======================================================================
# 2. 학습용 / 테스트용 데이터 분리
# ======================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"   - 학습용 데이터 {len(X_train)}건, 테스트용 데이터 {len(X_test)}건 분리 완료.")

# ======================================================================
# 3. 랜덤 포레스트 모델 학습
# ======================================================================
print("\n2. 랜덤 포레스트 모델 학습을 시작합니다... (Colab Pro 고용량 RAM으로 실행 중)")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=20, min_samples_leaf=10)
rf_model.fit(X_train, y_train)
print("✅ 모델 학습 완료!")

# ======================================================================
# 4. 모델 성능 검증 (MAE 계산)
# ======================================================================
predictions = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)

print("\n✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")
print("✅ [성공] 모델 성능 검증 완료!")
print(f"   - 예측 평균 절대 오차 (MAE): {mae:.2f} 일")
print("✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨")

# ======================================================================
# 5. 학습된 모델과 컬럼 정보 저장
# ======================================================================
MODEL_PATH = os.path.join(DRIVE_PATH, 'models')
os.makedirs(MODEL_PATH, exist_ok=True)
model_filename = os.path.join(MODEL_PATH, 'lead_time_predictor.joblib')
joblib.dump(rf_model, model_filename)
print(f"\n✅ 학습된 AI 모델이 아래 경로에 저장되었습니다:\n   - {model_filename}")
columns_filename = os.path.join(MODEL_PATH, 'lead_time_model_columns.joblib')
joblib.dump(X.columns, columns_filename)
print(f"\n✅ 모델 컬럼 정보가 아래 경로에 저장되었습니다:\n   - {columns_filename}")

In [ ]:
최종 마스터 버전 정리

서버 재기동 후 통합 수행 소스 아래 부터 시작 (마스터 수행코드)

In [12]:
# ======================================================================
# 1. 환경 설정 및 라이브러리 설치 (최신 버전 사용)
# ======================================================================
print("1. 전체 환경 설정을 시작합니다...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception: pass

# !! 중요 !! : 버전을 고정하지 않고, 호환되는 최신 라이브러리를 설치합니다.
print("\n필수 라이브러리를 최신 버전으로 설치합니다...")
!pip install pandas chromadb sentence-transformers gradio scikit-learn lightgbm --upgrade -q
print("✅ 라이브러리 설치 완료!")

# !! 중요 !! : 라이브러리 설치가 끝난 후, 모든 import를 시작합니다.
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import gradio as gr
import numpy as np
import os
import shutil
import traceback
from IPython.display import display, HTML
import stat
import joblib
from sklearn.feature_extraction import FeatureHasher
import lightgbm as lgb

# ======================================================================
# 2. 데이터 로드, DB 구축 및 모델 준비
# ======================================================================
print("\n2. 데이터 로드 및 AI 모델 준비를 시작합니다...")

# --- 경로 설정 ---
DRIVE_PATH = '/content/drive/MyDrive/ai_poc_project/'
ORIGINAL_DATA_PATH = os.path.join(DRIVE_PATH, 'original_data')
MODEL_PATH = os.path.join(DRIVE_PATH, 'models')
ZIPCODE_DATA_PATH = os.path.join(DRIVE_PATH, 'zipcode_data')

# --- 필요 데이터 로드 ---
try:
    print("\n원본 데이터들을 로드합니다...")
    enriched_df = pd.read_csv(os.path.join(ORIGINAL_DATA_PATH, 'item_data_enriched.csv'))
    orders_df_raw = pd.read_csv(os.path.join(ORIGINAL_DATA_PATH, 'orders_final.txt'), sep='\t')
    delivery_df_raw = pd.read_csv(os.path.join(ORIGINAL_DATA_PATH, 'delivery_data_cleaned.txt'), sep='\t')
    zipcode_db_df = pd.read_csv(os.path.join(ZIPCODE_DATA_PATH, 'all_zipcodes.csv'))
    print("✅ 모든 소스 데이터 로드 완료.")

    # ======== 수정된 부분: '품목코드' 타입을 '문자열'로 강제 통일 ========
    enriched_df['품목코드'] = enriched_df['품목코드'].astype(str)
    orders_df_raw['품목코드'] = orders_df_raw['품목코드'].astype(str)
    delivery_df_raw['품목코드'] = delivery_df_raw['품목코드'].astype(str)
    print(" - 모든 '품목코드' 타입을 문자열(str)로 통일했습니다.")
    # ==========================================================

except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")

# --- PoC #1 데이터 정합성 필터링 및 그룹화 ---
print(" - PoC #1용 데이터를 처리합니다...")
try:
    valid_item_codes = set(enriched_df['품목코드'].unique())
    orders_df = orders_df_raw[orders_df_raw['품목코드'].isin(valid_item_codes)].copy()
    department_rules = {
        '연구/개발': ['개발', 'Lab', 'R&D', '연구', '설계', 'S/W', 'AI', 'Solution', 'Platform', 'Tech', '기술', '선행', '메카', 'Digital'], '영업/마케팅': ['영업', '마케팅', 'Retail', 'Sales', 'PM', 'Biz'],
        '품질/검증': ['품질', 'QA', 'QC', '검사', '신뢰성', '보증', 'Validation', 'Compliance'], '인사/총무': ['인사', '총무', '채용', '교육', 'HR', '지원', '상생', 'ER'],
        '생산/공정': ['생산', '제조', 'FAB', '공정', 'PJT', 'P/J', '라인', 'Line', 'CELL', 'Plant', '정제', '충전', '전처리'], '구매/자재': ['구매', '자재', 'Spare'], '물류': ['물류', 'SCM'],
        '재무/회계': ['재무', '경리', '회계', '자금', 'IR', '결산'], '기획/전략': ['기획', '전략', '혁신'], '디자인': ['디자인', 'UX', 'UI'], 'CS/서비스': ['CS', '서비스', '홈케어'],
        '안전/환경': ['안전', '환경', 'EHS', '보건', '방재'], '지점/지역': ['지점', '지국', '총국', '센터'], '설비/인프라': ['설비', 'GAS', 'FT', '공무', '소방', 'Infra', '전기']
    }
    company_keywords = ['(주)', '㈜', '코리아', '삼성', 'SK', 'CJ', '현대', 'LG', '롯데']
    def group_team_name(team_name):
        if not isinstance(team_name, str): return '기타'
        team_name_lower = team_name.lower()
        for group, keywords in department_rules.items():
            for keyword in keywords:
                if keyword.lower() in team_name_lower: return group
        for keyword in company_keywords:
            if keyword in team_name:
                clean_name = team_name.split('(')[0].strip(); return clean_name if clean_name else team_name
        return '기타' # Revert change to '기 기타'
    orders_df['부서그룹'] = orders_df['주문자부서명'].apply(group_team_name)
    orders_df['주문수량'] = pd.to_numeric(orders_df['주문수량'], errors='coerce')
    orders_df.dropna(subset=['주문수량'], inplace=True)
    orders_df['주문수량'] = orders_df['주문수량'].astype(int)

    poc1_data_ready = True # Add flag
    print(" - ✅ PoC #1용 주문 데이터 처리 완료.")
except Exception as e:
    poc1_data_ready = False # Set flag to False on error
    print(f" - ⚠️ PoC #1 데이터 처리 실패: {e}")


# --- 인메모리(In-Memory) DB 구축 ---
print("\n - 인메모리 DB 구축을 시작합니다... (약 5-7분 소요)")
client = chromadb.Client()
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# PoC #2 DB 생성
cols_to_embed = ['품목코드', '한글품명', '영문품명', '규격', '모델명', '제조원명', '네이버_카테고리1', '네이버_카테고리2', '네이버_카테고리3', '네이버_카테고리4', '브랜드', '최저가']
enriched_df['embedding_text'] = enriched_df[cols_to_embed].fillna('').astype(str).agg(' '.join, axis=1)
item_embeddings = model.encode(enriched_df['embedding_text'].tolist(), show_progress_bar=True)
item_collection = client.get_or_create_collection("item_embeddings")
item_collection.add(embeddings=item_embeddings.tolist(), metadatas=[{"품목코드": code} for code in enriched_df['품목코드'].tolist()], documents=enriched_df['embedding_text'].tolist(), ids=[str(code) for code in enriched_df['품목코드'].tolist()])
# PoC #1 DB 생성 (Only if data was processed successfully)
if poc1_data_ready: # Check flag
    print("   - PoC #1 'team_embeddings' 생성 중...")
    team_purchase_list = orders_df.loc[orders_df.index.repeat(orders_df['주문수량'])]
    team_embedding_text = team_purchase_list.groupby('부서그룹')['품목코드'].apply(lambda x: ' '.join(x.astype(str)))
    team_df = team_embedding_text.reset_index()
    team_embeddings = model.encode(team_df['품목코드'].tolist(), show_progress_bar=True)
    team_collection = client.get_or_create_collection("team_embeddings")
    team_collection.add(embeddings=team_embeddings.tolist(), documents=team_df['부서그룹'].tolist(), ids=team_df['부서그룹'].tolist())
    print(f" - ✅ DB 구축 완료!")
else:
    print(" - ⚠️ PoC #1 데이터 처리 실패로 'team_embeddings' DB 구축을 건너뛰었습니다.")


# --- PoC#3 AI 모델 로드 및 데이터 전처리 ---
# --- AI 모델 로드 ---
try:
    print("\nAI 모델을 로드합니다...")
    # 우리가 마지막으로 성공시킨 LightGBM 모델과 그에 맞는 Hasher를 로드합니다.
    lead_time_model = joblib.load(os.path.join(MODEL_PATH, 'lead_time_predictor_lgbm.joblib'))
    hasher = joblib.load(os.path.join(MODEL_PATH, 'lead_time_hasher_final.joblib'))
    print("✅ PoC #3 예측 모델(LightGBM) 및 Hasher 로드 완료.")
except Exception as e:
    lead_time_model, hasher = None, None
    print(f"⚠️ PoC #3 모델 로드 실패: {e}")
# =================================================

# --- PoC #3 데이터 전처리 (별도의 try 블록으로 이동) ---
print("\n - PoC #3용 배송 데이터를 전처리합니다...")
try:
    delivery_df_processed = delivery_df_raw.copy()
    delivery_df_processed['우편번호'] = delivery_df_processed['우편번호'].astype(str)
    zipcode_db_df['우편번호'] = zipcode_db_df['우편번호'].astype(str)
    delivery_df_processed['운송지시일자'] = pd.to_datetime(delivery_df_processed['운송지시일자'], format='%Y%m%d', errors='coerce')
    delivery_df_processed['운송완료일자'] = pd.to_datetime(delivery_df_processed['운송완료일자'], format='%Y%m%d', errors='coerce')
    delivery_df_processed['리드타임'] = (delivery_df_processed['운송완료일자'] - delivery_df_processed['운송지시일자']).dt.days
    delivery_df_processed['주문_월'] = delivery_df_processed['운송지시일자'].dt.month
    delivery_df_processed['주문_요일'] = delivery_df_processed['운송지시일자'].dt.dayofweek
    zipcode_map = zipcode_db_df[['우편번호', '시도']].rename(columns={'시도': '도시'})
    delivery_df_processed = pd.merge(delivery_df_processed, zipcode_map, on='우편번호', how='left')
    delivery_df_processed.rename(columns={'납품물류유형': '배송유형'}, inplace=True)
    delivery_df_processed.dropna(subset=['품목코드', '배송유형', '도시', '리드타임', '주문_월', '주문_요일'], inplace=True)
    delivery_df_processed = delivery_df_processed[delivery_df_processed['리드타임'] >= 0]
    poc3_data_ready = True # Add flag
    print(" - ✅ PoC #3 준비 완료.")
except Exception as e:
    lead_time_model, hasher, delivery_df_processed = None, None, None
    poc3_data_ready = False # Set flag to False on error
    print(f" - ⚠️ PoC #3 준비 실패: {e}.")

print("\n✨ 모든 모델과 데이터가 메모리에 준비되었습니다.")

1. 전체 환경 설정을 시작합니다...
Mounted at /content/drive

필수 라이브러리를 최신 버전으로 설치합니다...
✅ 라이브러리 설치 완료!

2. 데이터 로드 및 AI 모델 준비를 시작합니다...

원본 데이터들을 로드합니다...
✅ 모든 소스 데이터 로드 완료.
 - 모든 '품목코드' 타입을 문자열(str)로 통일했습니다.
 - PoC #1용 데이터를 처리합니다...
 - ✅ PoC #1용 주문 데이터 처리 완료.

 - 인메모리 DB 구축을 시작합니다... (약 5-7분 소요)


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

   - PoC #1 'team_embeddings' 생성 중...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 - ✅ DB 구축 완료!

AI 모델을 로드합니다...
✅ PoC #3 예측 모델(LightGBM) 및 Hasher 로드 완료.

 - PoC #3용 배송 데이터를 전처리합니다...


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator FeatureHasher from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


 - ✅ PoC #3 준비 완료.

✨ 모든 모델과 데이터가 메모리에 준비되었습니다.


In [7]:
import pandas as pd
import numpy as np
import traceback
from IPython.display import display, HTML
import gradio as gr
from sentence_transformers import SentenceTransformer # 2단계에서 import했는지 확인

# ======================================================================
# 2. 모든 PoC 분석 함수 정의 (PoC #3 함수 수정)
# ======================================================================
print("2. 분석 함수 정의를 시작합니다...")

# (PoC #1, #2 함수는 이전과 동일)
def calculate_cosine_similarity(query_embedding, result_embeddings):
    query_vector = np.array(query_embedding); result_vectors = np.array(result_embeddings)
    query_norm = np.linalg.norm(query_vector)
    if query_norm == 0: return [0.0] * len(result_vectors)
    query_vector /= query_norm
    result_norms = np.linalg.norm(result_vectors, axis=1, keepdims=True)
    result_vectors /= np.where(result_norms == 0, 1, result_norms)
    return np.dot(result_vectors, query_vector).tolist()

def analyze_item_similarity(target_item_identifier: str):
    try:
        if not target_item_identifier: return "분석할 품목을 선택 또는 입력해주세요.", ""
        target_item_code = str(target_item_identifier.split(':')[0].strip()) if ':' in target_item_identifier else str(target_item_identifier.strip())
        selected_item_series = enriched_df[enriched_df['품목코드'].astype(str) == target_item_code].iloc[0]
        target_category = selected_item_series['네이버_카테고리1']
        target_embedding = item_collection.get(ids=[target_item_code], include=['embeddings'])['embeddings'][0]
        similar_items_result = item_collection.query(query_embeddings=[target_embedding], n_results=15, include=['metadatas', 'embeddings'])
        result_embeddings, result_metadatas = similar_items_result['embeddings'][0], similar_items_result['metadatas'][0]
        similarity_scores = calculate_cosine_similarity(target_embedding, result_embeddings)
        qualified_items = []
        for score, meta in zip(similarity_scores, result_metadatas):
            if str(meta['품목코드']) != target_item_code and score >= 0.75:
                item_details = enriched_df[enriched_df['품목코드'] == meta['품목코드']].iloc[0].to_dict()
                if item_details.get('네이버_카테고리1') == target_category:
                    item_details['유사도'] = score
                    qualified_items.append(item_details)
        if not qualified_items: recommended_df = pd.DataFrame()
        else: recommended_df = pd.DataFrame(qualified_items).sort_values(by='유사도', ascending=False).head(5)
        selected_item_html = f"""<h4>선택된 품목 정보</h4><div style="display:flex; align-items:center; border: 1px solid #E0E0E0; padding: 10px; border-radius: 5px;"><img src="{selected_item_series.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;"><div><b>{selected_item_series.get('한글품명', '')} ({selected_item_series.get('품목코드', '')})</b><br><b>카테고리:</b> {selected_item_series.get('네이버_카테고리1', '')} > {selected_item_series.get('네이버_카테고리2', '')}<br><b>규격:</b> {selected_item_series.get('규격', '')}</div></div>"""
        recommend_html = "<h4>AI 추천 유사 품목</h4>"
        if recommended_df.empty: recommend_html += f"<p>'{target_category}' 카테고리 내에서 기준점수(0.75)를 넘는 유사 품목을 찾을 수 없습니다.</p>"
        else:
            for _, row in recommended_df.iterrows():
                recommend_html += f"""<div style="display:flex; align-items:center; border: 1px solid #4CAF50; padding: 10px; border-radius: 5px; margin-bottom:10px;"><img src="{row.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;"><div><b>{row.get('한글품명', '')} ({row.get('품목코드', '')}) - <span style="color:#4CAF50; font-weight:bold;">유사도: {row.get('유사도', 0):.2f}</span></b><br><b>카테고리:</b> {row.get('네이버_카테고리1', '')} > {row.get('네이버_카테고리2', '')}<br><b>규격:</b> {row.get('규격', '')}</div></div>"""
        return selected_item_html, recommend_html
    except IndexError: return f"<h4>오류</h4><p>입력하신 품목코드 '{target_item_code}'를 찾을 수 없습니다.</p>", ""
    except Exception as e: return f"<h4>오류 발생</h4><p>{traceback.format_exc()}</p>", ""

### PoC #1: 유사 부서 상품 추천 함수 (HTML 출력으로 수정) ###
def analyze_team_preferences(target_team_group: str):
    """
    부서 그룹의 구매 랭킹을 HTML로 생성하여 반환하는 함수
    """
    try:
        if not target_team_group:
            return "분석할 부서 그룹을 선택해주세요."

        # 품목 상세 정보가 담긴 딕셔너리
        item_details_map = enriched_df.set_index('품목코드').to_dict('index')

        # 1. 선택한 부서 그룹의 Top 5 구매 품목 계산
        team_sales = orders_df[orders_df['부서그룹'] == target_team_group]
        if team_sales.empty:
            return f"**[{target_team_group}]** 그룹의 구매 데이터가 없습니다."

        top_items = team_sales.groupby('품목코드')['주문수량'].sum().nlargest(5).reset_index()

        ranking_html = f"<h4>**[{target_team_group}] 그룹 품목 구매 랭킹 (Top 5)**</h4>"

        for _, row in top_items.iterrows():
            item_code = row['품목코드']
            quantity = row['주문수량']

            # 상품 마스터(enriched_df)에서 품목 상세 정보 가져오기
            details = item_details_map.get(item_code, {})

            item_name = details.get('한글품명', '알 수 없는 품목')
            image_url = details.get('이미지_URL', '')
            category = f"{details.get('네이버_카테고리1', '')} > {details.get('네이버_카테고리2', '')}"
            spec = details.get('규격', '')
            price = details.get('최저가', 0)

            # HTML 생성
            ranking_html += f"""
            <div style="display:flex; align-items:center; border: 1px solid #E0E0E0; padding: 10px; border-radius: 5px; margin-bottom:10px;">
                <img src="{image_url}" width="100" style="margin-right:20px; border: 1px solid #DDD;">
                <div>
                    <b><span style="color:#007BFF; font-weight:bold;">{quantity}개</span> - [{item_code}] {item_name}</b><br>
                    <b>카테고리:</b> {category}<br>
                    <b>규격:</b> {spec}<br>
                    <b>최저가(참고):</b> {price:,}원
                </div>
            </div>
            """

        return ranking_html

    except Exception as e:
        return f"분석 중 오류 발생:\n{traceback.format_exc()}"

### PoC #3: 리드 타임 예측 함수 (키 오류 수정) ###
def predict_lead_time_poc3_all_cities(target_item_identifier: str):
    if lead_time_model is None or hasher is None:
        return "모델이 로드되지 않아 예측할 수 없습니다."
    if not target_item_identifier:
        return "분석할 품목을 선택 또는 입력해주세요."

    try:
        # --- 0. 기본 정보 준비 (모두 문자열로) ---
        target_item_code_str = str(target_item_identifier.split(':')[0].strip()) if ':' in target_item_identifier else str(target_item_identifier.strip())
        selected_item_series = enriched_df[enriched_df['품목코드'] == target_item_code_str].iloc[0]
        item_name = selected_item_series['한글품명']
        item_category = selected_item_series['네이버_카테고리1']

        prediction_text = f"### **[{item_name}]**의 전국 배송 예상 리드타임\n"
        prediction_text += "| 도시 | 예측 유형 | 배송유형 | 예상 소요일 |\n"
        prediction_text += "|:---|:---|:---|:---|\n"

        # --- 1. 유사/카테고리 품목 목록 미리 준비 (모두 문자열) ---
        target_embedding = item_collection.get(ids=[target_item_code_str], include=['embeddings'])['embeddings'][0]
        similar_items_result = item_collection.query(query_embeddings=[target_embedding], n_results=50)

        # ======== 수정된 부분: meta['id'] -> meta['품목코드'] ========
        similar_item_codes = [meta['품목코드'] for meta in similar_items_result['metadatas'][0] if meta['품목코드'] != target_item_code_str]
        # =========================================================

        category_item_codes = enriched_df[enriched_df['네이버_카테고리1'] == item_category]['품목코드'].unique()

        all_cities = sorted(delivery_df_processed['도시'].dropna().unique().tolist())

        for city_name in all_cities:
            has_result = False

            # --- 1순위: 실제 배송 이력 조회 (문자열로 비교) ---
            direct_history = delivery_df_processed[
                (delivery_df_processed['품목코드'] == target_item_code_str) &
                (delivery_df_processed['도시'] == city_name)
            ]
            used_delivery_types = direct_history['배송유형'].unique().tolist()

            if used_delivery_types:
                for delivery_type in used_delivery_types:
                    input_data = {'품목코드': target_item_code_str, '배송유형': delivery_type, '도시': city_name, '주문_월': str(pd.Timestamp.now().month), '주문_요일': str(pd.Timestamp.now().dayofweek)}
                    input_hashed = hasher.transform([input_data])
                    predicted_days = lead_time_model.predict(input_hashed)[0]
                    prediction_text += f"| {city_name} | **실제 이력 기반** | {delivery_type} | **{max(0, predicted_days):.1f}일** |\n"
                    has_result = True

            else:
                # --- 2순위: 유사 상품 배송 이력 조회 (문자열로 비교) ---
                similar_history = delivery_df_processed[
                    (delivery_df_processed['품목코드'].isin(similar_item_codes)) &
                    (delivery_df_processed['도시'] == city_name)
                ]
                if not similar_history.empty:
                    avg_lead_time = similar_history['리드타임'].mean()
                    prediction_text += f"| {city_name} | *유사 상품 기준* | - | *평균 {avg_lead_time:.1f}일* |\n"
                    has_result = True

                else:
                    # --- 3순위: 동일 카테고리 기준 조회 (문자열로 비교) ---
                    category_history = delivery_df_processed[
                        (delivery_df_processed['품목코드'].isin(category_item_codes)) &
                        (delivery_df_processed['도시'] == city_name)
                    ]
                    if not category_history.empty:
                        avg_lead_time = category_history['리드타임'].mean()
                        prediction_text += f"| {city_name} | *카테고리 기준* | - | *평균 {avg_lead_time:.1f}일* |\n"
                        has_result = True

            # 4순위: 예측 불가
            if not has_result:
                prediction_text += f"| {city_name} | 정보 없음 | - | - |\n"

        return prediction_text

    except IndexError:
        return f"<h4>오류</h4><p>입력하신 품목코드 '{target_item_code_str}'를 찾을 수 없습니다.</p>"
    except Exception as e:
        return f"예측 중 오류가 발생했습니다: {e}\n\n{traceback.format_exc()}"

print("✅ 모든 분석 함수 정의 완료.")
print("\n✨ 2단계 성공! 다음 3단계(UI 실행) 코드를 실행해주세요.")



2. 분석 함수 정의를 시작합니다...
✅ 모든 분석 함수 정의 완료.

✨ 2단계 성공! 다음 3단계(UI 실행) 코드를 실행해주세요.


In [10]:
import gradio as gr
import numpy as np
import traceback
from IPython.display import display, HTML
import pandas as pd

# ======================================================================
# 3. 모든 PoC 분석 함수 및 UI 정의
# ======================================================================
print("3. 분석 함수 정의 및 UI 실행을 시작합니다...")

# --- 분석 함수 정의 ---
def calculate_cosine_similarity(query_embedding, result_embeddings):
    query_vector = np.array(query_embedding); result_vectors = np.array(result_embeddings)
    query_norm = np.linalg.norm(query_vector)
    if query_norm == 0: return [0.0] * len(result_vectors)
    query_vector /= query_norm
    result_norms = np.linalg.norm(result_vectors, axis=1, keepdims=True)
    result_vectors /= np.where(result_norms == 0, 1, result_norms)
    return np.dot(result_vectors, query_vector).tolist()

def analyze_item_similarity(target_item_identifier: str):
    try:
        if not target_item_identifier: return "분석할 품목을 선택 또는 입력해주세요.", ""
        target_item_code_str = str(target_item_identifier.split(':')[0].strip()) if ':' in target_item_identifier else str(target_item_identifier.strip())
        selected_item_series = enriched_df[enriched_df['품목코드'] == target_item_code_str].iloc[0]
        target_category = selected_item_series['네이버_카테고리1']
        target_embedding = item_collection.get(ids=[target_item_code_str], include=['embeddings'])['embeddings'][0]
        similar_items_result = item_collection.query(query_embeddings=[target_embedding], n_results=15, include=['metadatas', 'embeddings'])
        result_embeddings, result_metadatas = similar_items_result['embeddings'][0], similar_items_result['metadatas'][0]
        similarity_scores = calculate_cosine_similarity(target_embedding, result_embeddings)
        qualified_items = []
        for score, meta in zip(similarity_scores, result_metadatas):
            if meta['id'] != target_item_code_str and score >= 0.75:
                item_details = enriched_df[enriched_df['품목코드'] == meta['id']].iloc[0].to_dict()
                if item_details.get('네이버_카테고리1') == target_category:
                    item_details['유사도'] = score
                    qualified_items.append(item_details)
        if not qualified_items: recommended_df = pd.DataFrame()
        else: recommended_df = pd.DataFrame(qualified_items).sort_values(by='유사도', ascending=False).head(5)
        selected_item_html = f"""<h4>선택된 품목 정보</h4><div style="display:flex; align-items:center; border: 1px solid #E0E0E0; padding: 10px; border-radius: 5px;"><img src="{selected_item_series.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;"><div><b>{selected_item_series.get('한글품명', '')} ({selected_item_series.get('품목코드', '')})</b><br><b>카테고리:</b> {selected_item_series.get('네이버_카테고리1', '')} > {selected_item_series.get('네이버_카테고리2', '')}<br><b>규격:</b> {selected_item_series.get('규격', '')}</div></div>"""
        recommend_html = "<h4>AI 추천 유사 품목</h4>"
        if recommended_df.empty: recommend_html += f"<p>'{target_category}' 카테고리 내에서 기준점수(0.75)를 넘는 유사 품목을 찾을 수 없습니다.</p>"
        else:
            for _, row in recommended_df.iterrows():
                recommend_html += f"""<div style="display:flex; align-items:center; border: 1px solid #4CAF50; padding: 10px; border-radius: 5px; margin-bottom:10px;"><img src="{row.get('이미지_URL', '')}" width="100" style="margin-right:20px; border: 1px solid #DDD;"><div><b>{row.get('한글품명', '')} ({row.get('품목코드', '')}) - <span style="color:#4CAF50; font-weight:bold;">유사도: {row.get('유사도', 0):.2f}</span></b><br><b>카테고리:</b> {row.get('네이버_카테고리1', '')} > {row.get('네이버_카테고리2', '')}<br><b>규격:</b> {row.get('규격', '')}</div></div>"""
        return selected_item_html, recommend_html
    except IndexError: return f"<h4>오류</h4><p>입력하신 품목코드 '{target_item_code_str}'를 찾을 수 없습니다.</p>", ""
    except Exception as e: return f"<h4>오류 발생</h4><p>{traceback.format_exc()}</p>", ""

def analyze_team_preferences(target_team_group: str):
    try:
        if not target_team_group: return "분석할 부서 그룹을 선택해주세요."
        item_code_to_name_map = pd.Series(enriched_df.한글품명.values, index=enriched_df.품목코드).to_dict()
        team_sales = orders_df[orders_df['부서그룹'] == target_team_group]
        if team_sales.empty: return f"**[{target_team_group}]** 그룹의 구매 데이터가 없습니다."
        top_items = team_sales.groupby('품목코드')['주문수량'].sum().nlargest(5).reset_index()
        top_items['품목명'] = top_items['품목코드'].map(item_code_to_name_map)
        ranking_text = f"**[{target_team_group}] 그룹 품목 구매 랭킹 (Top 5)**\n"
        for _, row in top_items.iterrows():
            ranking_text += f"- [{row.get('품목코드')}] {row.get('품목명', '이름없음')} ({row.get('주문수량', 0)}개)\n"
        return ranking_text
    except Exception as e: return f"분석 중 오류 발생:\n{traceback.format_exc()}"

def predict_lead_time_poc3(item_name, city_name):
    if lead_time_model is None or hasher is None: return "모델이 로드되지 않아 예측할 수 없습니다."
    if not item_name or not city_name: return "품목과 도시를 모두 선택해주세요."
    try:
        item_code_str = enriched_df.loc[enriched_df['한글품명'] == item_name, '품목코드'].iloc[0]
        prediction_text = f"### **[{item_name}]**의 **[{city_name}]** 예상 리드타임\n"
        for delivery_type in ['배송', '직납', '집배송']:
            input_data = {'품목코드': item_code_str, '배송유형': delivery_type, '도시': city_name, '주문_월': str(pd.Timestamp.now().month), '주문_요일': str(pd.Timestamp.now().dayofweek)}
            input_hashed = hasher.transform([input_data])
            predicted_days = lead_time_model.predict(input_hashed)[0]
            prediction_text += f"- **{delivery_type}**: {max(0, predicted_days):.1f}일 소요 예상\n"
        target_embedding = item_collection.get(ids=[item_code_str], include=['embeddings'])['embeddings'][0]
        similar_items_result = item_collection.query(query_embeddings=[target_embedding], n_results=50)
        similar_item_codes = [meta['id'] for meta in similar_items_result['metadatas'][0] if meta['id'] != item_code_str]
        similar_history = delivery_df_processed[(delivery_df_processed['품목코드'].isin(similar_item_codes)) & (delivery_df_processed['도시'] == city_name)]
        if not similar_history.empty:
            avg_lead_time = similar_history['리드타임'].mean()
            prediction_text += f"\n---\n*참고: 이 도시로 배송된 **유사 상품들은 평균 {avg_lead_time:.1f}일**이 소요되었습니다.*"
        else:
            item_category = enriched_df.loc[enriched_df['품목코드'] == item_code_str, '네이버_카테고리1'].iloc[0]
            category_item_codes = enriched_df[enriched_df['네이버_카테고리1'] == item_category]['품목코드'].unique()
            category_history = delivery_df_processed[(delivery_df_processed['품목코드'].isin(category_item_codes)) & (delivery_df_processed['도시'] == city_name)]
            if not category_history.empty:
                avg_lead_time = category_history['리드타임'].mean()
                prediction_text += f"\n---\n*참고: 이 도시의 **동일 카테고리 상품들은 평균 {avg_lead_time:.1f}일**이 소요되었습니다.*"
            else:
                prediction_text += "\n---\n*참고: 이 도시로 배송된 유사/카테고리 상품 이력이 없습니다.*"
        return prediction_text
    except Exception as e: return f"예측 중 오류 발생: {e}\n\n{traceback.format_exc()}"

# --- Gradio UI 준비 및 실행 ---
print("\n3. Gradio UI를 실행합니다...")
teams_list = orders_df['부서그룹'].value_counts().index.tolist()
categories_list = sorted(enriched_df['네이버_카테고리1'].dropna().unique().tolist())
enriched_df['item_display'] = enriched_df['품목코드'] + " : " + enriched_df['한글품명']
all_cities_list = sorted(delivery_df_processed['도시'].dropna().unique().tolist()) if 'delivery_df_processed' in locals() and delivery_df_processed is not None else []
all_items_list = sorted(enriched_df['한글품명'].dropna().unique().tolist())
def update_item_dropdown(category):
    if not category: return gr.Dropdown(choices=[])
    filtered_df = enriched_df[enriched_df['네이버_카테고리1'] == category]
    choices = sorted(filtered_df['item_display'].dropna().unique().tolist())
    return gr.Dropdown(choices=choices, value=choices[0] if choices else None)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 IMK Plus AI PoC Project (통합)")
    with gr.Tabs():
        with gr.TabItem("PoC #1: 유사 부서 상품 추천"):
            gr.Markdown("특정 부서 그룹을 선택하면, 해당 그룹의 품목 구매 랭킹 Top 5를 보여줍니다.")
            with gr.Row():
                team_dropdown = gr.Dropdown(choices=teams_list, label="분석할 부서 그룹을 선택하세요")
                poc1_button = gr.Button("분석 실행", variant="primary")
            ranking_output = gr.Textbox(label="선택한 그룹의 품목 구매 랭킹", lines=7)
            poc1_button.click(fn=analyze_team_preferences, inputs=team_dropdown, outputs=ranking_output)
        with gr.TabItem("PoC #2: 품목별 유사 상품 추천"):
            gr.Markdown("AI가 **유사도 0.75 이상**인 품목 중 가장 유사한 **최대 5개**를 추천합니다.")
            with gr.Tabs():
                with gr.TabItem("카테고리로 찾기"):
                    with gr.Row():
                        category_dd = gr.Dropdown(choices=categories_list, label="1. 대분류 카테고리를 선택하세요")
                        item_dd = gr.Dropdown(label="2. 품목을 선택하세요 (카테고리 먼저 선택)", interactive=True)
                    cat_button = gr.Button("유사 품목 찾기", variant="primary")
                with gr.TabItem("직접 검색하기"):
                    item_code_tb = gr.Textbox(label="품목코드를 직접 입력하세요", placeholder="예: 1419128400")
                    code_button = gr.Button("유사 품목 찾기", variant="primary")
            gr.HTML("<hr style='margin: 20px 0;'>")
            selected_item_output = gr.HTML()
            similar_items_output = gr.HTML()
            category_dd.change(fn=update_item_dropdown, inputs=category_dd, outputs=item_dd)
            cat_button.click(fn=analyze_item_similarity, inputs=item_dd, outputs=[selected_item_output, similar_items_output])
            code_button.click(fn=analyze_item_similarity, inputs=item_code_tb, outputs=[selected_item_output, similar_items_output])
        with gr.TabItem("PoC #3: 배송 리드타임 예측"):
            gr.Markdown("품목을 선택하면, AI가 **전국 도시별** 예상 배송 소요 시간을 '폭포수 모델'로 예측합니다.")
            with gr.Tabs():
                with gr.TabItem("카테고리로 찾기"):
                    with gr.Row():
                        poc3_category_dd = gr.Dropdown(choices=categories_list, label="1. 대분류 카테고리를 선택하세요")
                        poc3_item_dd = gr.Dropdown(label="2. 품목을 선택하세요 (카테고리 먼저 선택)", interactive=True)
                    poc3_cat_button = gr.Button("전국 리드타임 조회", variant="primary")
                with gr.TabItem("직접 검색하기"):
                    poc3_item_code_tb = gr.Textbox(label="품목코드를 직접 입력하세요", placeholder="예: 1419128400")
                    poc3_code_button = gr.Button("전국 리드타임 조회", variant="primary")
            gr.HTML("<hr style='margin: 20px 0;'>")
            poc3_output = gr.Markdown()

            poc3_category_dd.change(fn=update_item_dropdown, inputs=poc3_category_dd, outputs=poc3_item_dd)
            poc3_cat_button.click(fn=predict_lead_time_poc3_all_cities, inputs=poc3_item_dd, outputs=poc3_output)
            poc3_code_button.click(fn=predict_lead_time_poc3_all_cities, inputs=poc3_item_code_tb, outputs=poc3_output)

demo.launch(share=True, debug=True)

3. 분석 함수 정의 및 UI 실행을 시작합니다...

3. Gradio UI를 실행합니다...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8c3d1723281398377a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8c3d1723281398377a.gradio.live


In [ ]:
import pandas as pd

print("PoC #3 시연에 적합한, '배송지가 다양한' 상위 5개 품목을 찾습니다...")
print("(모든 품목코드를 '문자열'로 통일하여 검증합니다)")

try:
    # --- 1. 모든 품목코드를 '문자열'로 통일 ---
    enriched_df['품목코드'] = enriched_df['품목코드'].astype(str)
    delivery_df_processed['품목코드'] = delivery_df_processed['품목코드'].astype(str)

    # --- 2. '진짜' 품목코드(상품 마스터에 있는) 기록만 필터링 ---
    valid_item_codes = set(enriched_df['품목코드'].unique())
    filtered_delivery_df = delivery_df_processed[
        delivery_df_processed['품목코드'].isin(valid_item_codes)
    ]

    # --- 3. 품목별 고유 도시 '수' 계산 ---
    city_diversity_count = filtered_delivery_df.groupby('품목코드')['도시'].nunique().sort_values(ascending=False)

    # --- 4. 품목별 고유 도시 '목록' 생성 (신규 추가) ---
    city_diversity_list = filtered_delivery_df.groupby('품목코드')['도시'].apply(lambda x: list(x.unique()))

    # --- 5. 품목명 결합 ---
    item_names_df = enriched_df[['품목코드', '한글품명']]

    # --- 6. (도시 수) + (도시 목록) + (품목명) 데이터 결합 ---
    demo_candidates = pd.merge(
        city_diversity_count.reset_index(name='배송된_도시_수'),
        city_diversity_list.reset_index(name='배송된_도시_목록'),
        on='품목코드',
        how='left'
    )
    demo_candidates = pd.merge(
        demo_candidates,
        item_names_df,
        on='품목코드',
        how='left'
    )

    # --- 7. 최종 결과 출력 ---
    print("\n[PoC #3 시연 추천 품목 Top 5 (상세 도시 목록 포함)]")
    # '품목코드', '한글품명', '배송된_도시_수', '배송된_도시_목록' 순서로 정렬
    final_columns = ['품목코드', '한글품명', '배송된_도시_수', '배송된_도시_목록']
    display(demo_candidates[final_columns].head(5))

except NameError:
    print("\n❌ [에러] 데이터가 메모리에 없습니다.")
    print("   - '1&2단계 통합 코드'를 먼저 실행해주세요.")
except Exception as e:
    print(f"\n❌ [에러] 분석 중 오류가 발생했습니다: {e}")